In [181]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, KNNImputer, IterativeImputer
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', '{:,.2f}'.format)

In [182]:
# credit_data=pd.read_csv("./01_dataset.csv")

credit_data=pd.read_csv(r"C:\Users\user\Documents\python\round2\Bảng ĐH_Vòng 2_DSTC 2024_Dataset\01_dataset.csv")

In [183]:
credit_data.head(10)

,customer_id,label,SHORT_TERM_COUNT,MID_TERM_COUNT,LONG_TERM_COUNT,SHORT_TERM_COUNT_BANK,MID_TERM_COUNT_BANK,LONG_TERM_COUNT_BANK,SHORT_TERM_COUNT_NON_BANK,MID_TERM_COUNT_NON_BANK,LONG_TERM_COUNT_NON_BANK,NUMBER_OF_LOANS,NUMBER_OF_LOANS_BANK,NUMBER_OF_LOANS_NON_BANK,NUMBER_OF_CREDIT_CARDS,NUMBER_OF_CREDIT_CARDS_BANK,NUMBER_OF_CREDIT_CARDS_NON_BANK,NUMBER_OF_RELATIONSHIP,NUMBER_OF_RELATIONSHIP_BANK,NUMBER_OF_RELATIONSHIP_NON_BANK,NUM_NEW_LOAN_TAKEN_3M,NUM_NEW_LOAN_TAKEN_6M,NUM_NEW_LOAN_TAKEN_9M,NUM_NEW_LOAN_TAKEN_12M,NUM_NEW_LOAN_TAKEN_BANK_3M,NUM_NEW_LOAN_TAKEN_BANK_6M,NUM_NEW_LOAN_TAKEN_BANK_9M,NUM_NEW_LOAN_TAKEN_BANK_12M,NUM_NEW_LOAN_TAKEN_NON_BANK_3M,NUM_NEW_LOAN_TAKEN_NON_BANK_6M,NUM_NEW_LOAN_TAKEN_NON_BANK_9M,NUM_NEW_LOAN_TAKEN_NON_BANK_12M,OUTSTANDING_BAL_LOAN_CURRENT,OUTSTANDING_BAL_LOAN_3M,OUTSTANDING_BAL_LOAN_6M,OUTSTANDING_BAL_LOAN_9M,OUTSTANDING_BAL_LOAN_12M,OUTSTANDING_BAL_CC_3M,OUTSTANDING_BAL_CC_6M,OUTSTANDING_BAL_CC_9M,OUTSTANDING_BAL_CC_12M,OUTSTANDING_BAL_ALL_3M,OUTSTANDING_BAL_ALL_6M,OUTSTANDING_BAL_ALL_9M,OUTSTANDING_BAL_ALL_12M,OUTSTANDING_BAL_LOAN_3M_6M,OUTSTANDING_BAL_LOAN_6M_9M,OUTSTANDING_BAL_LOAN_9M_12M,OUTSTANDING_BAL_LOAN_6M_12M,OUTSTANDING_BAL_LOAN_3M_12M,OUTSTANDING_BAL_CC_3M_6M,OUTSTANDING_BAL_CC_6M_9M,OUTSTANDING_BAL_CC_9M_12M,OUTSTANDING_BAL_CC_6M_12M,OUTSTANDING_BAL_CC_3M_12M,OUTSTANDING_BAL_ALL_3M_6M,OUTSTANDING_BAL_ALL_6M_9M,OUTSTANDING_BAL_ALL_9M_12M,OUTSTANDING_BAL_ALL_6M_12M,OUTSTANDING_BAL_ALL_3M_12M,INCREASING_BAL_3M_LOAN,INCREASING_BAL_6M_LOAN,INCREASING_BAL_3M_CC,INCREASING_BAL_6M_CC,INCREASING_BAL_3M_ALL,INCREASING_BAL_6M_ALL,OUTSTANDING_BAL_CC_CURRENT,CREDIT_CARD_MONTH_SINCE_10DPD,CREDIT_CARD_MONTH_SINCE_30DPD,CREDIT_CARD_MONTH_SINCE_60DPD,CREDIT_CARD_MONTH_SINCE_90DPD,CREDIT_CARD_NUMBER_OF_LATE_PAYMENT,ENQUIRIES_3M,ENQUIRIES_6M,ENQUIRIES_9M,ENQUIRIES_12M,ENQUIRIES_FROM_BANK_3M,ENQUIRIES_FROM_NON_BANK_3M,ENQUIRIES_FOR_LOAN_3M,ENQUIRIES_FOR_CC_3M,ENQUIRIES_FROM_BANK_FOR_LOAN_3M,ENQUIRIES_FROM_NON_BANK_FOR_LOAN_3M,ENQUIRIES_FROM_BANK_FOR_CC_3M,ENQUIRIES_FROM_NON_BANK_FOR_CC_3M,ENQUIRIES_FROM_BANK_6M,ENQUIRIES_FROM_NON_BANK_6M,ENQUIRIES_FOR_LOAN_6M,ENQUIRIES_FOR_CC_6M,ENQUIRIES_FROM_BANK_FOR_LOAN_6M,ENQUIRIES_FROM_NON_BANK_FOR_LOAN_6M,ENQUIRIES_FROM_BANK_FOR_CC_6M,ENQUIRIES_FROM_NON_BANK_FOR_CC_6M,ENQUIRIES_FROM_BANK_9M,ENQUIRIES_FROM_NON_BANK_9M,ENQUIRIES_FOR_LOAN_9M,ENQUIRIES_FOR_CC_9M,ENQUIRIES_FROM_BANK_FOR_LOAN_9M,ENQUIRIES_FROM_NON_BANK_FOR_LOAN_9M,ENQUIRIES_FROM_BANK_FOR_CC_9M,ENQUIRIES_FROM_NON_BANK_FOR_CC_9M,ENQUIRIES_FROM_BANK_12M,ENQUIRIES_FROM_NON_BANK_12M,ENQUIRIES_FOR_LOAN_12M,ENQUIRIES_FOR_CC_12M,ENQUIRIES_FROM_BANK_FOR_LOAN_12M,ENQUIRIES_FROM_NON_BANK_FOR_LOAN_12M,ENQUIRIES_FROM_BANK_FOR_CC_12M,ENQUIRIES_FROM_NON_BANK_FOR_CC_12M,ENQUIRIES_3M_6M,ENQUIRIES_6M_9M,ENQUIRIES_9M_12M,ENQUIRIES_6M_12M,ENQUIRIES_3M_12M,ENQUIRIES_FROM_BANK_3M_6M,ENQUIRIES_FROM_BANK_6M_9M,ENQUIRIES_FROM_BANK_9M_12M,ENQUIRIES_FROM_BANK_6M_12M,ENQUIRIES_FROM_BANK_3M_12M,ENQUIRIES_FROM_NON_BANK_3M_6M,ENQUIRIES_FROM_NON_BANK_6M_9M,ENQUIRIES_FROM_NON_BANK_9M_12M,ENQUIRIES_FROM_NON_BANK_6M_12M,ENQUIRIES_FROM_NON_BANK_3M_12M,OUTSTANDING_BAL_ALL_CURRENT
0,1639,1,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,NaN,1.00,1.00,1.00,1.00,1.00,1.00,1.00,NaN,1.00,"1,000,000.00","1,000,000.00","1,000,000.00","1,000,000.00","1,000,205.00","1,000,000.00","1,000,000.00","1,000,000.00",NaN,"1,000,000.00","1,000,000.00","1,000,000.00","1,000,205.00","1,000,000.00","1,000,000.00","1,000,000.00","1,000,000.00","1,000,000.00","1,000,000.00","1,000,000.00","1,000,000.00","1,000,000.00","1,000,000.00","1,000,000.00","1,000,000.00","1,000,000.00","1,000,000.00","1,000,000.00","1,000,000.00","1,000,000.00","1,000,000.00","1,000,000.00","1,000,000.00",NaN,"1,000,000.00",431.00,431.00,431.00,431.00,1.00,7.00,7.00,17.00,17.00,2.00,7.00,7.00,2.00,2.00,7.00,2.00,2.00,2.00,7.00,7.00,2.00,2.00,7.00,2.00,NaN,2.00,17.00,12.00,7.00,2.00,12.00,2.00,7.00,2.00,17.00,12.00,7.00,2.00,12.00

In [184]:
credit_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Columns: 124 entries, customer_id to OUTSTANDING_BAL_ALL_CURRENT
dtypes: float64(122), int64(2)
memory usage: 18.9 MB


In [185]:
credit_data.shape

(20000, 124)

In [186]:
credit_data.columns.tolist()

['customer_id',
 'label',
 'SHORT_TERM_COUNT',
 'MID_TERM_COUNT',
 'LONG_TERM_COUNT',
 'SHORT_TERM_COUNT_BANK',
 'MID_TERM_COUNT_BANK',
 'LONG_TERM_COUNT_BANK',
 'SHORT_TERM_COUNT_NON_BANK',
 'MID_TERM_COUNT_NON_BANK',
 'LONG_TERM_COUNT_NON_BANK',
 'NUMBER_OF_LOANS',
 'NUMBER_OF_LOANS_BANK',
 'NUMBER_OF_LOANS_NON_BANK',
 'NUMBER_OF_CREDIT_CARDS',
 'NUMBER_OF_CREDIT_CARDS_BANK',
 'NUMBER_OF_CREDIT_CARDS_NON_BANK',
 'NUMBER_OF_RELATIONSHIP',
 'NUMBER_OF_RELATIONSHIP_BANK',
 'NUMBER_OF_RELATIONSHIP_NON_BANK',
 'NUM_NEW_LOAN_TAKEN_3M',
 'NUM_NEW_LOAN_TAKEN_6M',
 'NUM_NEW_LOAN_TAKEN_9M',
 'NUM_NEW_LOAN_TAKEN_12M',
 'NUM_NEW_LOAN_TAKEN_BANK_3M',
 'NUM_NEW_LOAN_TAKEN_BANK_6M',
 'NUM_NEW_LOAN_TAKEN_BANK_9M',
 'NUM_NEW_LOAN_TAKEN_BANK_12M',
 'NUM_NEW_LOAN_TAKEN_NON_BANK_3M',
 'NUM_NEW_LOAN_TAKEN_NON_BANK_6M',
 'NUM_NEW_LOAN_TAKEN_NON_BANK_9M',
 'NUM_NEW_LOAN_TAKEN_NON_BANK_12M',
 'OUTSTANDING_BAL_LOAN_CURRENT',
 'OUTSTANDING_BAL_LOAN_3M',
 'OUTSTANDING_BAL_LOAN_6M',
 'OUTSTANDING_BAL_LOAN_9M',


In [187]:
credit_data.describe()

,customer_id,label,SHORT_TERM_COUNT,MID_TERM_COUNT,LONG_TERM_COUNT,SHORT_TERM_COUNT_BANK,MID_TERM_COUNT_BANK,LONG_TERM_COUNT_BANK,SHORT_TERM_COUNT_NON_BANK,MID_TERM_COUNT_NON_BANK,LONG_TERM_COUNT_NON_BANK,NUMBER_OF_LOANS,NUMBER_OF_LOANS_BANK,NUMBER_OF_LOANS_NON_BANK,NUMBER_OF_CREDIT_CARDS,NUMBER_OF_CREDIT_CARDS_BANK,NUMBER_OF_CREDIT_CARDS_NON_BANK,NUMBER_OF_RELATIONSHIP,NUMBER_OF_RELATIONSHIP_BANK,NUMBER_OF_RELATIONSHIP_NON_BANK,NUM_NEW_LOAN_TAKEN_3M,NUM_NEW_LOAN_TAKEN_6M,NUM_NEW_LOAN_TAKEN_9M,NUM_NEW_LOAN_TAKEN_12M,NUM_NEW_LOAN_TAKEN_BANK_3M,NUM_NEW_LOAN_TAKEN_BANK_6M,NUM_NEW_LOAN_TAKEN_BANK_9M,NUM_NEW_LOAN_TAKEN_BANK_12M,NUM_NEW_LOAN_TAKEN_NON_BANK_3M,NUM_NEW_LOAN_TAKEN_NON_BANK_6M,NUM_NEW_LOAN_TAKEN_NON_BANK_9M,NUM_NEW_LOAN_TAKEN_NON_BANK_12M,OUTSTANDING_BAL_LOAN_CURRENT,OUTSTANDING_BAL_LOAN_3M,OUTSTANDING_BAL_LOAN_6M,OUTSTANDING_BAL_LOAN_9M,OUTSTANDING_BAL_LOAN_12M,OUTSTANDING_BAL_CC_3M,OUTSTANDING_BAL_CC_6M,OUTSTANDING_BAL_CC_9M,OUTSTANDING_BAL_CC_12M,OUTSTANDING_BAL_ALL_3M,OUTSTANDING_BAL_ALL_6M,OUTSTANDING_BAL_ALL_9M,OUTSTANDING_BAL_ALL_12M,OUTSTANDING_BAL_LOAN_3M_6M,OUTSTANDING_BAL_LOAN_6M_9M,OUTSTANDING_BAL_LOAN_9M_12M,OUTSTANDING_BAL_LOAN_6M_12M,OUTSTANDING_BAL_LOAN_3M_12M,OUTSTANDING_BAL_CC_3M_6M,OUTSTANDING_BAL_CC_6M_9M,OUTSTANDING_BAL_CC_9M_12M,OUTSTANDING_BAL_CC_6M_12M,OUTSTANDING_BAL_CC_3M_12M,OUTSTANDING_BAL_ALL_3M_6M,OUTSTANDING_BAL_ALL_6M_9M,OUTSTANDING_BAL_ALL_9M_12M,OUTSTANDING_BAL_ALL_6M_12M,OUTSTANDING_BAL_ALL_3M_12M,INCREASING_BAL_3M_LOAN,INCREASING_BAL_6M_LOAN,INCREASING_BAL_3M_CC,INCREASING_BAL_6M_CC,INCREASING_BAL_3M_ALL,INCREASING_BAL_6M_ALL,OUTSTANDING_BAL_CC_CURRENT,CREDIT_CARD_MONTH_SINCE_10DPD,CREDIT_CARD_MONTH_SINCE_30DPD,CREDIT_CARD_MONTH_SINCE_60DPD,CREDIT_CARD_MONTH_SINCE_90DPD,CREDIT_CARD_NUMBER_OF_LATE_PAYMENT,ENQUIRIES_3M,ENQUIRIES_6M,ENQUIRIES_9M,ENQUIRIES_12M,ENQUIRIES_FROM_BANK_3M,ENQUIRIES_FROM_NON_BANK_3M,ENQUIRIES_FOR_LOAN_3M,ENQUIRIES_FOR_CC_3M,ENQUIRIES_FROM_BANK_FOR_LOAN_3M,ENQUIRIES_FROM_NON_BANK_FOR_LOAN_3M,ENQUIRIES_FROM_BANK_FOR_CC_3M,ENQUIRIES_FROM_NON_BANK_FOR_CC_3M,ENQUIRIES_FROM_BANK_6M,ENQUIRIES_FROM_NON_BANK_6M,ENQUIRIES_FOR_LOAN_6M,ENQUIRIES_FOR_CC_6M,ENQUIRIES_FROM_BANK_FOR_LOAN_6M,ENQUIRIES_FROM_NON_BANK_FOR_LOAN_6M,ENQUIRIES_FROM_BANK_FOR_CC_6M,ENQUIRIES_FROM_NON_BANK_FOR_CC_6M,ENQUIRIES_FROM_BANK_9M,ENQUIRIES_FROM_NON_BANK_9M,ENQUIRIES_FOR_LOAN_9M,ENQUIRIES_FOR_CC_9M,ENQUIRIES_FROM_BANK_FOR_LOAN_9M,ENQUIRIES_FROM_NON_BANK_FOR_LOAN_9M,ENQUIRIES_FROM_BANK_FOR_CC_9M,ENQUIRIES_FROM_NON_BANK_FOR_CC_9M,ENQUIRIES_FROM_BANK_12M,ENQUIRIES_FROM_NON_BANK_12M,ENQUIRIES_FOR_LOAN_12M,ENQUIRIES_FOR_CC_12M,ENQUIRIES_FROM_BANK_FOR_LOAN_12M,ENQUIRIES_FROM_NON_BANK_FOR_LOAN_12M,ENQUIRIES_FROM_BANK_FOR_CC_12M,ENQUIRIES_FROM_NON_BANK_FOR_CC_12M,ENQUIRIES_3M_6M,ENQUIRIES_6M_9M,ENQUIRIES_9M_12M,ENQUIRIES_6M_12M,ENQUIRIES_3M_12M,ENQUIRIES_FROM_BANK_3M_6M,ENQUIRIES_FROM_BANK_6M_9M,ENQUIRIES_FROM_BANK_9M_12M,ENQUIRIES_FROM_BANK_6M_12M,ENQUIRIES_FROM_BANK_3M_12M,ENQUIRIES_FROM_NON_BANK_3M_6M,ENQUIRIES_FROM_NON_BANK_6M_9M,ENQUIRIES_FROM_NON_BANK_9M_12M,ENQUIRIES_FROM_NON_BANK_6M_12M,ENQUIRIES_FROM_NON_BANK_3M_12M,OUTSTANDING_BAL_ALL_CURRENT
count,"20,000.00","20,000.00","18,000.00","18,000.00","18,000.00","18,000.00","18,000.00","18,000.00","18,000.00","18,000.00","18,000.00","18,000.00","18,000.00","18,000.00","18,000.00","18,000.00","18,000.00","18,000.00","18,000.00","18,000.00","18,000.00","18,000.00","18,000.00","18,000.00","18,000.00","18,000.00","18,000.00","18,000.00","18,000.00","18,000.00","18,000.00","18,000.00","18,000.00","18,000.00","18,000.00","18,000.00","18,000.00","18,000.00","18,000.00","18,000.00","18,000.00","18,000.00","18,000.00","18,000.00","18,000.00","18,000.00","18,000.00","18,000.00","18,000.00","18,000.00","18,000.00","18,000.00","18,000.00","18,000.00","18,000.00","18,000.00","18,000.00","18,000.00","18,000.00","18,000.00","18,000.00","18,000.00","18,000.00","18,000.00","18,000.00","18,000.00","18,000.00","18,000.00","18,000.00","18,000.00","18,000.00","18,000.00"

In [188]:
credit_data.isnull().sum()

customer_id                          0
label                                0
SHORT_TERM_COUNT                  2000
MID_TERM_COUNT                    2000
LONG_TERM_COUNT                   2000
                                  ... 
ENQUIRIES_FROM_NON_BANK_6M_9M     2000
ENQUIRIES_FROM_NON_BANK_9M_12M    2000
ENQUIRIES_FROM_NON_BANK_6M_12M    2000
ENQUIRIES_FROM_NON_BANK_3M_12M    2000
OUTSTANDING_BAL_ALL_CURRENT       2000
Length: 124, dtype: int64

In [189]:
label_0=credit_data[credit_data["label"]==0]
label_1=credit_data[credit_data["label"]==1]
n=len(label_0)/len(credit_data["label"])
n_1=len(label_1)/len(credit_data["label"])
print(f"Số khách hàng trả nợ đúng hạn {len(label_0)}")
print(f"Số khách hàng quá hạn trả nợ {len(label_1)}")
print(f"Tỷ lệ số khách hàng trả nợ đúng hạn trên tổng số khách hàng: {n}")
print(f"Tỷ lệ số khách hàng quá hạn trả nợ trên tổng số khách hàng: {n_1}")

Số khách hàng trả nợ đúng hạn 16356
Số khách hàng quá hạn trả nợ 3644
Tỷ lệ số khách hàng trả nợ đúng hạn trên tổng số khách hàng: 0.8178
Tỷ lệ số khách hàng quá hạn trả nợ trên tổng số khách hàng: 0.1822


In [190]:
# remaining_columns = credit_data.drop(columns=["customer_id", "label"]).columns.tolist()
# for c in remaining_columns:
#     sns.boxplot(x=c, data=credit_data)
#     plt.title(f"Boxplot of {c}")
#     plt.show()

In [191]:
# remaining_columns

In [192]:
credit_data['SHORT_TERM_COUNT'] = credit_data['SHORT_TERM_COUNT'].fillna(credit_data['SHORT_TERM_COUNT_BANK'] + credit_data['SHORT_TERM_COUNT_NON_BANK']-1)

In [193]:
credit_data['SHORT_TERM_COUNT'].isna().sum()

np.int64(360)

In [194]:
credit_data['SHORT_TERM_COUNT_BANK'] = credit_data['SHORT_TERM_COUNT_BANK'].fillna(credit_data['SHORT_TERM_COUNT'] - credit_data['SHORT_TERM_COUNT_NON_BANK'] +1)
credit_data['SHORT_TERM_COUNT_NON_BANK'] = credit_data['SHORT_TERM_COUNT_NON_BANK'].fillna(credit_data['SHORT_TERM_COUNT'] - credit_data['SHORT_TERM_COUNT_BANK'] +1)

In [195]:
credit_data['SHORT_TERM_COUNT'].isna().sum(), credit_data['SHORT_TERM_COUNT_BANK'].isna().sum(), credit_data['SHORT_TERM_COUNT_NON_BANK'].isna().sum()

(np.int64(360), np.int64(368), np.int64(382))

In [196]:
nan_count_per_row = credit_data[['SHORT_TERM_COUNT', 'SHORT_TERM_COUNT_BANK', 'SHORT_TERM_COUNT_NON_BANK']].isna().sum(axis=1)
rows_with_2_or_more_nan = (nan_count_per_row >= 2).sum()
print(f"Số lượng dòng có từ 2 giá trị NaN trở lên: {rows_with_2_or_more_nan}")
rows_with_3_or_more_nan = (nan_count_per_row >= 3).sum()
print(f"Số lượng dòng có từ 3 giá trị NaN: {rows_with_3_or_more_nan}")

Số lượng dòng có từ 2 giá trị NaN trở lên: 542
Số lượng dòng có từ 3 giá trị NaN: 26


In [197]:
credit_data['MID_TERM_COUNT'] = credit_data['MID_TERM_COUNT'].fillna(credit_data['MID_TERM_COUNT_BANK'] + credit_data['MID_TERM_COUNT_NON_BANK']-1)
credit_data['MID_TERM_COUNT_BANK'] = credit_data['MID_TERM_COUNT_BANK'].fillna(credit_data['MID_TERM_COUNT'] - credit_data['MID_TERM_COUNT_NON_BANK'] +1)
credit_data['MID_TERM_COUNT_NON_BANK'] = credit_data['MID_TERM_COUNT_NON_BANK'].fillna(credit_data['MID_TERM_COUNT'] - credit_data['MID_TERM_COUNT_BANK'] +1)

credit_data['MID_TERM_COUNT'].isna().sum(), credit_data['MID_TERM_COUNT_BANK'].isna().sum(), credit_data['MID_TERM_COUNT_NON_BANK'].isna().sum()

(np.int64(385), np.int64(363), np.int64(366))

In [198]:
nan_count_per_row = credit_data[['MID_TERM_COUNT', 'MID_TERM_COUNT_BANK', 'MID_TERM_COUNT_NON_BANK']].isna().sum(axis=1)
rows_with_2_or_more_nan = (nan_count_per_row >= 2).sum()
print(f"Số lượng dòng có từ 2 giá trị NaN trở lên: {rows_with_2_or_more_nan}")
rows_with_3_or_more_nan = (nan_count_per_row >= 3).sum()
print(f"Số lượng dòng có từ 3 giá trị NaN: {rows_with_3_or_more_nan}")

Số lượng dòng có từ 2 giá trị NaN trở lên: 545
Số lượng dòng có từ 3 giá trị NaN: 24


In [199]:
credit_data['LONG_TERM_COUNT'] = credit_data['LONG_TERM_COUNT'].fillna(credit_data['LONG_TERM_COUNT_BANK'] + credit_data['LONG_TERM_COUNT_NON_BANK']-1)
credit_data['LONG_TERM_COUNT_BANK'] = credit_data['LONG_TERM_COUNT_BANK'].fillna(credit_data['LONG_TERM_COUNT'] - credit_data['LONG_TERM_COUNT_NON_BANK'] +1)
credit_data['LONG_TERM_COUNT_NON_BANK'] = credit_data['LONG_TERM_COUNT_NON_BANK'].fillna(credit_data['LONG_TERM_COUNT'] - credit_data['LONG_TERM_COUNT_BANK'] +1)

credit_data['LONG_TERM_COUNT'].isna().sum(), credit_data['LONG_TERM_COUNT_BANK'].isna().sum(), credit_data['LONG_TERM_COUNT_NON_BANK'].isna().sum()

(np.int64(402), np.int64(392), np.int64(399))

In [200]:
nan_count_per_row = credit_data[['LONG_TERM_COUNT', 'LONG_TERM_COUNT_BANK', 'LONG_TERM_COUNT_NON_BANK']].isna().sum(axis=1)
rows_with_2_or_more_nan = (nan_count_per_row >= 2).sum()
print(f"Số lượng dòng có từ 2 giá trị NaN trở lên: {rows_with_2_or_more_nan}")
rows_with_3_or_more_nan = (nan_count_per_row >= 3).sum()
print(f"Số lượng dòng có từ 3 giá trị NaN: {rows_with_3_or_more_nan}")

Số lượng dòng có từ 2 giá trị NaN trở lên: 588
Số lượng dòng có từ 3 giá trị NaN: 17


In [201]:
credit_data['NUMBER_OF_LOANS'] = credit_data['NUMBER_OF_LOANS'].fillna(credit_data['NUMBER_OF_LOANS_BANK'] + credit_data['NUMBER_OF_LOANS_NON_BANK']-1)
credit_data['NUMBER_OF_LOANS_BANK'] = credit_data['NUMBER_OF_LOANS_BANK'].fillna(credit_data['NUMBER_OF_LOANS'] - credit_data['NUMBER_OF_LOANS_NON_BANK'] +1)
credit_data['NUMBER_OF_LOANS_NON_BANK'] = credit_data['NUMBER_OF_LOANS_NON_BANK'].fillna(credit_data['NUMBER_OF_LOANS'] - credit_data['NUMBER_OF_LOANS_BANK'] +1)

credit_data['NUMBER_OF_LOANS'].isna().sum(), credit_data['NUMBER_OF_LOANS_BANK'].isna().sum(), credit_data['NUMBER_OF_LOANS_NON_BANK'].isna().sum()

(np.int64(373), np.int64(395), np.int64(397))

In [202]:
nan_count_per_row = credit_data[['NUMBER_OF_LOANS', 'NUMBER_OF_LOANS_BANK', 'NUMBER_OF_LOANS_NON_BANK']].isna().sum(axis=1)
rows_with_2_or_more_nan = (nan_count_per_row >= 2).sum()
print(f"Số lượng dòng có từ 2 giá trị NaN trở lên: {rows_with_2_or_more_nan}")
rows_with_3_or_more_nan = (nan_count_per_row >= 3).sum()
print(f"Số lượng dòng có từ 3 giá trị NaN: {rows_with_3_or_more_nan}")

Số lượng dòng có từ 2 giá trị NaN trở lên: 572
Số lượng dòng có từ 3 giá trị NaN: 21


In [203]:
credit_data['NUMBER_OF_RELATIONSHIP'] = credit_data['NUMBER_OF_RELATIONSHIP'].fillna(credit_data['NUMBER_OF_RELATIONSHIP_BANK'] + credit_data['NUMBER_OF_RELATIONSHIP_NON_BANK']-1)
credit_data['NUMBER_OF_RELATIONSHIP_BANK'] = credit_data['NUMBER_OF_RELATIONSHIP_BANK'].fillna(credit_data['NUMBER_OF_RELATIONSHIP'] - credit_data['NUMBER_OF_RELATIONSHIP_NON_BANK'] +1)
credit_data['NUMBER_OF_RELATIONSHIP_NON_BANK'] = credit_data['NUMBER_OF_RELATIONSHIP_NON_BANK'].fillna(credit_data['NUMBER_OF_RELATIONSHIP'] - credit_data['NUMBER_OF_RELATIONSHIP_BANK'] +1)

credit_data['NUMBER_OF_RELATIONSHIP'].isna().sum(), credit_data['NUMBER_OF_RELATIONSHIP_BANK'].isna().sum(), credit_data['NUMBER_OF_RELATIONSHIP_NON_BANK'].isna().sum()

(np.int64(383), np.int64(386), np.int64(395))

In [204]:
nan_count_per_row = credit_data[['NUMBER_OF_RELATIONSHIP', 'NUMBER_OF_RELATIONSHIP_BANK', 'NUMBER_OF_RELATIONSHIP_NON_BANK']].isna().sum(axis=1)
rows_with_1_or_more_nan = (nan_count_per_row == 1).sum()
print(f"Số lượng dòng có 1 giá trị NaN: {rows_with_1_or_more_nan}")
rows_with_2_or_more_nan = (nan_count_per_row == 2).sum()
print(f"Số lượng dòng có 2 giá trị NaN: {rows_with_2_or_more_nan}")
rows_with_3_or_more_nan = (nan_count_per_row == 3).sum()
print(f"Số lượng dòng có 3 giá trị NaN: {rows_with_3_or_more_nan}")

Số lượng dòng có 1 giá trị NaN: 0
Số lượng dòng có 2 giá trị NaN: 552
Số lượng dòng có 3 giá trị NaN: 20


In [205]:
def fill_nan_for_loans(df, periods):
    for period in periods:
        loan_col = f'NUM_NEW_LOAN_TAKEN_{period}M'
        bank_col = f'NUM_NEW_LOAN_TAKEN_BANK_{period}M'
        non_bank_col = f'NUM_NEW_LOAN_TAKEN_NON_BANK_{period}M'
        
        
        df[loan_col] = df[loan_col].fillna(df[bank_col].fillna(df[non_bank_col]))
        df[bank_col] = df[bank_col].fillna(df[loan_col].fillna(df[non_bank_col]))
        df[non_bank_col] = df[non_bank_col].fillna(df[bank_col].fillna(df[bank_col]))
    
    

periods = [3, 6, 9, 12]

fill_nan_for_loans(credit_data, periods)

In [206]:
def print_nan_for_loans(df, periods):
    for period in periods:
        loan_col = f'NUM_NEW_LOAN_TAKEN_{period}M'
        bank_col = f'NUM_NEW_LOAN_TAKEN_BANK_{period}M'
        non_bank_col = f'NUM_NEW_LOAN_TAKEN_NON_BANK_{period}M'
        
        nan_count_per_row = df[[loan_col, bank_col, non_bank_col]].isna().sum(axis=1)
        rows_with_1 = (nan_count_per_row == 1).sum()
        print(f"Số lượng dòng có 1 giá trị NaN trong {period}M: {rows_with_1}")
        rows_with_2 = (nan_count_per_row == 2).sum()
        print(f"Số lượng dòng có 2 giá trị NaN trong {period}M: {rows_with_2}")
        rows_with_3 = (nan_count_per_row == 3).sum()
        print(f"Số lượng dòng có 3 giá trị NaN trong {period}M: {rows_with_3}")
        
    

periods = [3, 6, 9, 12]

print_nan_for_loans(credit_data, periods)

Số lượng dòng có 1 giá trị NaN trong 3M: 0
Số lượng dòng có 2 giá trị NaN trong 3M: 0
Số lượng dòng có 3 giá trị NaN trong 3M: 15
Số lượng dòng có 1 giá trị NaN trong 6M: 0
Số lượng dòng có 2 giá trị NaN trong 6M: 0
Số lượng dòng có 3 giá trị NaN trong 6M: 24
Số lượng dòng có 1 giá trị NaN trong 9M: 0
Số lượng dòng có 2 giá trị NaN trong 9M: 0
Số lượng dòng có 3 giá trị NaN trong 9M: 32
Số lượng dòng có 1 giá trị NaN trong 12M: 0
Số lượng dòng có 2 giá trị NaN trong 12M: 0
Số lượng dòng có 3 giá trị NaN trong 12M: 20


In [207]:
def fill_nan_for_outstanding_bal(row, periods):
    for period in periods:
        bal_all = f'OUTSTANDING_BAL_ALL_{period}M'
        bal_loan = f'OUTSTANDING_BAL_LOAN_{period}M'
        bal_cc = f'OUTSTANDING_BAL_CC_{period}M'
        
        if row[bal_cc] == 1000000 and row[bal_loan] == 1000000: 
            row[bal_all] = row[bal_all] if pd.notna(row[bal_all]) else row[bal_loan]
        
        elif row[bal_all] == 1000000 and row[bal_loan] == 1000000:
            row[bal_cc] = row[bal_cc] if pd.notna(row[bal_cc]) else row[bal_loan]
        
        if row[bal_loan] == 1000000 and row[bal_cc] > 1000000:
            row[bal_all] = row[bal_all] if pd.notna(row[bal_all]) else row[bal_cc]
        
        if row[bal_loan] > 1000000 and row[bal_cc] > 1000000 and pd.isna(row[bal_all]):
            row[bal_all] = row[bal_loan] + (row[bal_cc] - 1000000)
        elif row[bal_loan] > 1000000 and pd.isna(row[bal_cc]) and row[bal_all] > 1000000:
            row[bal_cc] = row[bal_all] - (row[bal_loan] - 1000000)
        elif pd.isna(row[bal_loan]) and row[bal_cc] > 1000000 and row[bal_all] > 1000000:
            row[bal_loan] = (row[bal_all] - row[bal_cc] - 1000000)
        elif row[bal_loan] > 1000000 and row[bal_cc] < 1000000 and pd.isna(row[bal_all]):
            row[bal_all] = row[bal_loan]
        elif pd.isna(row[bal_loan]) and row[bal_cc] < 1000000 and row[bal_all] > 1000000:
            row[bal_loan] = row[bal_all]

    return row

periods = [3, 6, 9, 12]
credit_data = credit_data.apply(fill_nan_for_outstanding_bal, axis=1, periods=periods)


In [208]:

# columns_to_check = [
#     'OUTSTANDING_BAL_ALL_3M', 'OUTSTANDING_BAL_LOAN_3M', 'OUTSTANDING_BAL_CC_3M',
#     'OUTSTANDING_BAL_ALL_6M', 'OUTSTANDING_BAL_LOAN_6M', 'OUTSTANDING_BAL_CC_6M',
#     'OUTSTANDING_BAL_ALL_9M', 'OUTSTANDING_BAL_LOAN_9M', 'OUTSTANDING_BAL_CC_9M',
#     'OUTSTANDING_BAL_ALL_12M', 'OUTSTANDING_BAL_LOAN_12M', 'OUTSTANDING_BAL_CC_12M'
# ]

def rows_with_one_nan(df, columns):
    nan_count_per_row = df[columns].isna().sum(axis=1)
    rows_with_one_nan = df[nan_count_per_row == 1]
    return rows_with_one_nan

columns_to_check = ['OUTSTANDING_BAL_ALL_3M', 'OUTSTANDING_BAL_LOAN_3M', 'OUTSTANDING_BAL_CC_3M']

rows_with_exactly_one_nan = rows_with_one_nan(credit_data, columns_to_check)

rows_with_exactly_one_nan[columns_to_check].head()

,OUTSTANDING_BAL_ALL_3M,OUTSTANDING_BAL_LOAN_3M,OUTSTANDING_BAL_CC_3M
11,NaN,"1,000,056.67","1,000,000.00"
30,NaN,"1,000,006.67","1,000,000.00"
129,"1,000,000.00",NaN,"1,000,000.00"
137,"1,000,070.00",NaN,"1,000,000.00"
204,NaN,"1,000,120.00","1,000,000.00"


đoạn code check dưới này chưa đúng số vì chưa tìm ra được cách tính số giá trị nan chỉ trong 3 cột. thực tế số sẽ nhỏ hơn nhiều so với kết quả bên dưới...

In [209]:
def print_nan_for_outstanding_bal(df, periods):
    for period in periods:
        bal_all = f'OUTSTANDING_BAL_ALL_{period}M'
        bal_loan = f'OUTSTANDING_BAL_LOAN_{period}M'
        bal_cc = f'OUTSTANDING_BAL_CC_{period}M'
        columns_to_check = [bal_all, bal_loan, bal_cc]
        nan_count_per_row = df[columns_to_check].isna().sum(axis=1)
        rows_with_1 = (nan_count_per_row == 1).sum()
        print(f"Số lượng dòng có 1 giá trị NaN trong {period}M: {rows_with_1}")
        rows_with_2 = (nan_count_per_row == 2).sum()
        print(f"Số lượng dòng có 2 giá trị NaN trong {period}M: {rows_with_2}")
        rows_with_3 = (nan_count_per_row == 3).sum()
        print(f"Số lượng dòng có 3 giá trị NaN trong {period}M: {rows_with_3}")

periods = [3, 6, 9, 12]

print_nan_for_outstanding_bal(credit_data, periods)

Số lượng dòng có 1 giá trị NaN trong 3M: 683
Số lượng dòng có 2 giá trị NaN trong 3M: 518
Số lượng dòng có 3 giá trị NaN trong 3M: 31
Số lượng dòng có 1 giá trị NaN trong 6M: 649
Số lượng dòng có 2 giá trị NaN trong 6M: 504
Số lượng dòng có 3 giá trị NaN trong 6M: 20
Số lượng dòng có 1 giá trị NaN trong 9M: 618
Số lượng dòng có 2 giá trị NaN trong 9M: 513
Số lượng dòng có 3 giá trị NaN trong 9M: 16
Số lượng dòng có 1 giá trị NaN trong 12M: 654
Số lượng dòng có 2 giá trị NaN trong 12M: 529
Số lượng dòng có 3 giá trị NaN trong 12M: 21


In [210]:
def fill_nan_for_increasing_bal(row, periods):
    for period in periods:
        bal_all = f'INCREASING_BAL_{period}M_ALL'
        bal_loan = f'INCREASING_BAL_{period}M_LOAN'
        bal_cc = f'INCREASING_BAL_{period}M_CC'
        
        if pd.notna(row[bal_cc]) and pd.notna(row[bal_loan]):
            if row[bal_cc] == 1000000 and row[bal_loan] == 1000000:
                row[bal_all] = row[bal_all] if pd.notna(row[bal_all]) else row[bal_loan]
            elif row[bal_cc] == 1000010 and row[bal_loan] == 1000000:
                row[bal_all] = row[bal_all] if pd.notna(row[bal_all]) else row[bal_cc]
            elif row[bal_cc] == 1000000 and row[bal_loan] == 1000010:
                row[bal_all] = row[bal_all] if pd.notna(row[bal_all]) else row[bal_loan]
            elif row[bal_cc] == 1000010 and row[bal_loan] == 1000010:
                row[bal_all] = row[bal_all] if pd.notna(row[bal_all]) else row[bal_loan]
                
        if pd.notna(row[bal_cc]) and row[bal_cc] <= 1000000 and pd.isna(row[bal_all]):
            row[bal_all] = row[bal_loan]
        elif pd.isna(row[bal_cc]) and pd.notna(row[bal_all]) and pd.notna(row[bal_loan]):
            row[bal_cc] = row[bal_all]
        elif pd.isna(row[bal_loan]) and pd.notna(row[bal_cc]) and pd.notna(row[bal_all]):
            row[bal_loan] = row[bal_all]
        
    return row

periods = [3, 6]
credit_data = credit_data.apply(fill_nan_for_increasing_bal, axis=1, periods=periods)


In [211]:
def fill_nan_for_increasing_bal(df, periods):
    for period in periods:
        bal_all = f'INCREASING_BAL_{period}M_ALL'
        bal_loan = f'INCREASING_BAL_{period}M_LOAN'
        bal_cc = f'INCREASING_BAL_{period}M_CC'
        
        nan_count_per_row = df[[bal_all, bal_loan, bal_cc]].isna().sum(axis=1)
        rows_with_1 = (nan_count_per_row == 1).sum()
        print(f"Số lượng dòng có 1 giá trị NaN trong {period}M: {rows_with_1}")
        rows_with_2 = (nan_count_per_row == 2).sum()
        print(f"Số lượng dòng có 2 giá trị NaN trong {period}M: {rows_with_2}")
        rows_with_3 = (nan_count_per_row == 3).sum()
        print(f"Số lượng dòng có 3 giá trị NaN trong {period}M: {rows_with_3}")
        

periods = [3, 6]

fill_nan_for_increasing_bal(credit_data, periods)

Số lượng dòng có 1 giá trị NaN trong 3M: 0
Số lượng dòng có 2 giá trị NaN trong 3M: 510
Số lượng dòng có 3 giá trị NaN trong 3M: 19
Số lượng dòng có 1 giá trị NaN trong 6M: 0
Số lượng dòng có 2 giá trị NaN trong 6M: 504
Số lượng dòng có 3 giá trị NaN trong 6M: 24


In [212]:
def fill_nan_for_enquiries(df, periods):
    for period in periods:
        enq_col = f'ENQUIRIES_{period}M'
        enq_bank_col = f'ENQUIRIES_FROM_BANK_{period}M'
        enq_non_bank_col = f'ENQUIRIES_FROM_NON_BANK_{period}M'
        
        
        df[enq_col] = df[enq_col].fillna(df[enq_bank_col] + df[enq_non_bank_col] -2)
        df[enq_bank_col] = df[enq_bank_col].fillna(df[enq_col] - df[enq_non_bank_col] +2)
        df[enq_non_bank_col] = df[enq_non_bank_col].fillna(df[enq_col] - df[enq_bank_col] +2)
    
periods = [3, 6, 9, 12]

fill_nan_for_enquiries(credit_data, periods)

In [213]:
def print_nan_for_enquiries(df, periods):
    for period in periods:
        loan_col = f'ENQUIRIES_{period}M'
        bank_col = f'ENQUIRIES_FROM_BANK_{period}M'
        non_bank_col = f'ENQUIRIES_FROM_NON_BANK_{period}M'
        
        nan_count_per_row = df[[loan_col, bank_col, non_bank_col]].isna().sum(axis=1)
        rows_with_1 = (nan_count_per_row == 1).sum()
        print(f"Số lượng dòng có 1 giá trị NaN trong {period}M: {rows_with_1}")
        rows_with_2 = (nan_count_per_row == 2).sum()
        print(f"Số lượng dòng có 2 giá trị NaN trong {period}M: {rows_with_2}")
        rows_with_3 = (nan_count_per_row == 3).sum()
        print(f"Số lượng dòng có 3 giá trị NaN trong {period}M: {rows_with_3}")

periods = [3, 6, 9, 12]

print_nan_for_enquiries(credit_data, periods)

Số lượng dòng có 1 giá trị NaN trong 3M: 0
Số lượng dòng có 2 giá trị NaN trong 3M: 574
Số lượng dòng có 3 giá trị NaN trong 3M: 20
Số lượng dòng có 1 giá trị NaN trong 6M: 0
Số lượng dòng có 2 giá trị NaN trong 6M: 554
Số lượng dòng có 3 giá trị NaN trong 6M: 18
Số lượng dòng có 1 giá trị NaN trong 9M: 0
Số lượng dòng có 2 giá trị NaN trong 9M: 544
Số lượng dòng có 3 giá trị NaN trong 9M: 17
Số lượng dòng có 1 giá trị NaN trong 12M: 0
Số lượng dòng có 2 giá trị NaN trong 12M: 513
Số lượng dòng có 3 giá trị NaN trong 12M: 13


In [214]:
def fill_nan_for_enquiries_loan(df, periods):
    for period in periods:
        enq_loan_col = f'ENQUIRIES_FOR_LOAN_{period}M'
        enq_bank_loan_col = f'ENQUIRIES_FROM_BANK_FOR_LOAN_{period}M'
        enq_non_bank_loan_col = f'ENQUIRIES_FROM_NON_BANK_FOR_LOAN_{period}M'
        
        
        df[enq_loan_col] = df[enq_loan_col].fillna(df[enq_bank_loan_col] + df[enq_non_bank_loan_col] -2)
        df[enq_bank_loan_col] = df[enq_bank_loan_col].fillna(df[enq_loan_col] - df[enq_non_bank_loan_col] +2)
        df[enq_non_bank_loan_col] = df[enq_non_bank_loan_col].fillna(df[enq_loan_col] - df[enq_bank_loan_col] +2)
    
periods = [3, 6, 9, 12]

fill_nan_for_enquiries_loan(credit_data, periods)

In [215]:
def print_nan_for_enquiries_loan(df, periods):
    for period in periods:
        enq_loan_col = f'ENQUIRIES_FOR_LOAN_{period}M'
        enq_bank_loan_col = f'ENQUIRIES_FROM_BANK_FOR_LOAN_{period}M'
        enq_non_bank_loan_col = f'ENQUIRIES_FROM_NON_BANK_FOR_LOAN_{period}M'
        
        nan_count_per_row = df[[enq_loan_col, enq_bank_loan_col, enq_non_bank_loan_col]].isna().sum(axis=1)
        rows_with_1 = (nan_count_per_row == 1).sum()
        print(f"Số lượng dòng có 1 giá trị NaN trong {period}M: {rows_with_1}")
        rows_with_2 = (nan_count_per_row == 2).sum()
        print(f"Số lượng dòng có 2 giá trị NaN trong {period}M: {rows_with_2}")
        rows_with_3 = (nan_count_per_row == 3).sum()
        print(f"Số lượng dòng có 3 giá trị NaN trong {period}M: {rows_with_3}")

periods = [3, 6, 9, 12]

print_nan_for_enquiries_loan(credit_data, periods)

Số lượng dòng có 1 giá trị NaN trong 3M: 0
Số lượng dòng có 2 giá trị NaN trong 3M: 589
Số lượng dòng có 3 giá trị NaN trong 3M: 16
Số lượng dòng có 1 giá trị NaN trong 6M: 0
Số lượng dòng có 2 giá trị NaN trong 6M: 541
Số lượng dòng có 3 giá trị NaN trong 6M: 20
Số lượng dòng có 1 giá trị NaN trong 9M: 0
Số lượng dòng có 2 giá trị NaN trong 9M: 541
Số lượng dòng có 3 giá trị NaN trong 9M: 21
Số lượng dòng có 1 giá trị NaN trong 12M: 0
Số lượng dòng có 2 giá trị NaN trong 12M: 554
Số lượng dòng có 3 giá trị NaN trong 12M: 18


In [216]:
def fill_nan_for_enquiries_cc(df, periods):
    for period in periods:
        enq_cc_col = f'ENQUIRIES_FOR_CC_{period}M'
        enq_bank_cc_col = f'ENQUIRIES_FROM_BANK_FOR_CC_{period}M'
        enq_non_bank_cc_col = f'ENQUIRIES_FROM_NON_BANK_FOR_CC_{period}M'
        
        
        df[enq_cc_col] = df[enq_cc_col].fillna(df[enq_bank_cc_col] + df[enq_non_bank_cc_col] -2)
        df[enq_bank_cc_col] = df[enq_bank_cc_col].fillna(df[enq_cc_col] - df[enq_non_bank_cc_col] +2)
        df[enq_non_bank_cc_col] = df[enq_non_bank_cc_col].fillna(df[enq_cc_col] - df[enq_bank_cc_col] +2)
    
periods = [3, 6, 9, 12]

fill_nan_for_enquiries_cc(credit_data, periods)

In [217]:
def print_nan_for_enquiries_cc(df, periods):
    for period in periods:
        enq_cc_col = f'ENQUIRIES_FOR_CC_{period}M'
        enq_bank_cc_col = f'ENQUIRIES_FROM_BANK_FOR_CC_{period}M'
        enq_non_bank_cc_col = f'ENQUIRIES_FROM_NON_BANK_FOR_CC_{period}M'
        
        nan_count_per_row = df[[enq_cc_col, enq_bank_cc_col, enq_non_bank_cc_col]].isna().sum(axis=1)
        rows_with_1 = (nan_count_per_row == 1).sum()
        print(f"Số lượng dòng có 1 giá trị NaN trong {period}M: {rows_with_1}")
        rows_with_2 = (nan_count_per_row == 2).sum()
        print(f"Số lượng dòng có 2 giá trị NaN trong {period}M: {rows_with_2}")
        rows_with_3 = (nan_count_per_row == 3).sum()
        print(f"Số lượng dòng có 3 giá trị NaN trong {period}M: {rows_with_3}")

periods = [3, 6, 9, 12]

print_nan_for_enquiries_cc(credit_data, periods)

Số lượng dòng có 1 giá trị NaN trong 3M: 0
Số lượng dòng có 2 giá trị NaN trong 3M: 520
Số lượng dòng có 3 giá trị NaN trong 3M: 18
Số lượng dòng có 1 giá trị NaN trong 6M: 0
Số lượng dòng có 2 giá trị NaN trong 6M: 566
Số lượng dòng có 3 giá trị NaN trong 6M: 20
Số lượng dòng có 1 giá trị NaN trong 9M: 0
Số lượng dòng có 2 giá trị NaN trong 9M: 527
Số lượng dòng có 3 giá trị NaN trong 9M: 23
Số lượng dòng có 1 giá trị NaN trong 12M: 0
Số lượng dòng có 2 giá trị NaN trong 12M: 581
Số lượng dòng có 3 giá trị NaN trong 12M: 19


In [218]:
def fill_nan_for_enquiries_xM_xM(row, periods):
    for period in periods:
        enquiries_all = f'ENQUIRIES_{period}'
        enquiries_bank = f'ENQUIRIES_FROM_BANK_{period}'
        enquiries_non_bank = f'ENQUIRIES_FROM_NON_BANK_{period}'
                
        if pd.notna(row[enquiries_all]) and row[enquiries_bank] < 0:
            row[enquiries_non_bank] = row[enquiries_non_bank] if pd.notna(row[enquiries_non_bank]) else row[enquiries_all]
        if pd.notna(row[enquiries_non_bank]) and row[enquiries_bank] < 0:
            row[enquiries_all] = row[enquiries_all] if pd.notna(row[enquiries_all]) else row[enquiries_non_bank]
        

    return row

periods = ['3M_6M', '6M_9M', '9M_12M', '6M_12M', '3M_12M']

credit_data = credit_data.apply(fill_nan_for_enquiries_xM_xM, axis=1, periods=periods)


In [219]:
def print_nan_for_enquiries_xM_xM(df, periods):
    for period in periods:
        enquiries_all = f'ENQUIRIES_{period}'
        enquiries_bank = f'ENQUIRIES_FROM_BANK_{period}'
        enquiries_non_bank = f'ENQUIRIES_FROM_NON_BANK_{period}'
        
        nan_count_per_row = df[[enquiries_all, enquiries_bank, enquiries_non_bank]].isna().sum(axis=1)
        rows_with_1 = (nan_count_per_row == 1).sum()
        print(f"Số lượng dòng có 1 giá trị NaN trong {period}M: {rows_with_1}")
        rows_with_2 = (nan_count_per_row == 2).sum()
        print(f"Số lượng dòng có 2 giá trị NaN trong {period}M: {rows_with_2}")
        rows_with_3 = (nan_count_per_row == 3).sum()
        print(f"Số lượng dòng có 3 giá trị NaN trong {period}M: {rows_with_3}")

periods = ['3M_6M', '6M_9M', '9M_12M', '6M_12M', '3M_12M']

print_nan_for_enquiries_xM_xM(credit_data, periods)

Số lượng dòng có 1 giá trị NaN trong 3M_6MM: 3372
Số lượng dòng có 2 giá trị NaN trong 3M_6MM: 541
Số lượng dòng có 3 giá trị NaN trong 3M_6MM: 23
Số lượng dòng có 1 giá trị NaN trong 6M_9MM: 3519
Số lượng dòng có 2 giá trị NaN trong 6M_9MM: 539
Số lượng dòng có 3 giá trị NaN trong 6M_9MM: 21
Số lượng dòng có 1 giá trị NaN trong 9M_12MM: 3637
Số lượng dòng có 2 giá trị NaN trong 9M_12MM: 536
Số lượng dòng có 3 giá trị NaN trong 9M_12MM: 20
Số lượng dòng có 1 giá trị NaN trong 6M_12MM: 3617
Số lượng dòng có 2 giá trị NaN trong 6M_12MM: 514
Số lượng dòng có 3 giá trị NaN trong 6M_12MM: 23
Số lượng dòng có 1 giá trị NaN trong 3M_12MM: 3604
Số lượng dòng có 2 giá trị NaN trong 3M_12MM: 553
Số lượng dòng có 3 giá trị NaN trong 3M_12MM: 25


In [220]:
def fill_outstanding_bal(row):

    if pd.notna(row['OUTSTANDING_BAL_CC_CURRENT']) and pd.notna(row['OUTSTANDING_BAL_LOAN_CURRENT']):
        if row['OUTSTANDING_BAL_CC_CURRENT'] <= 1000000:
            row['OUTSTANDING_BAL_ALL_CURRENT'] = row['OUTSTANDING_BAL_ALL_CURRENT'] if pd.notna(row['OUTSTANDING_BAL_ALL_CURRENT']) else row['OUTSTANDING_BAL_LOAN_CURRENT']
        elif row['OUTSTANDING_BAL_LOAN_CURRENT'] == 1000000 and row['OUTSTANDING_BAL_CC_CURRENT'] == 1000000:
            row['OUTSTANDING_BAL_ALL_CURRENT'] = 1000000
        elif row['OUTSTANDING_BAL_LOAN_CURRENT'] > 1000000 and row['OUTSTANDING_BAL_CC_CURRENT'] > 1000000:
            row['OUTSTANDING_BAL_ALL_CURRENT'] = row['OUTSTANDING_BAL_ALL_CURRENT'] if pd.notna(row['OUTSTANDING_BAL_ALL_CURRENT']) else ((row['OUTSTANDING_BAL_LOAN_CURRENT'] - 1000000) + row['OUTSTANDING_BAL_CC_CURRENT'])

    if pd.notna(row['OUTSTANDING_BAL_CC_CURRENT']) and pd.notna(row['OUTSTANDING_BAL_ALL_CURRENT']):
        if row['OUTSTANDING_BAL_CC_CURRENT'] <= 1000000:
            row['OUTSTANDING_BAL_LOAN_CURRENT'] = row['OUTSTANDING_BAL_LOAN_CURRENT'] if pd.notna(row['OUTSTANDING_BAL_LOAN_CURRENT']) else row['OUTSTANDING_BAL_ALL_CURRENT']
        else:
            row['OUTSTANDING_BAL_LOAN_CURRENT'] = row['OUTSTANDING_BAL_LOAN_CURRENT'] if pd.notna(row['OUTSTANDING_BAL_LOAN_CURRENT']) else (row['OUTSTANDING_BAL_ALL_CURRENT'] - (row['OUTSTANDING_BAL_CC_CURRENT'] - 1000000))

    if pd.notna(row['OUTSTANDING_BAL_LOAN_CURRENT']) and pd.notna(row['OUTSTANDING_BAL_ALL_CURRENT']):
        if row['OUTSTANDING_BAL_LOAN_CURRENT'] == 1000000 and row['OUTSTANDING_BAL_ALL_CURRENT'] == 1000000:
            row['OUTSTANDING_BAL_CC_CURRENT'] = row['OUTSTANDING_BAL_CC_CURRENT'] if pd.notna(row['OUTSTANDING_BAL_CC_CURRENT']) else row['OUTSTANDING_BAL_ALL_CURRENT']
        elif row['OUTSTANDING_BAL_LOAN_CURRENT'] == row['OUTSTANDING_BAL_CC_CURRENT']:
            row['OUTSTANDING_BAL_CC_CURRENT'] = row['OUTSTANDING_BAL_CC_CURRENT'] if pd.notna(row['OUTSTANDING_BAL_CC_CURRENT']) else 1000000
        elif row['OUTSTANDING_BAL_LOAN_CURRENT'] > 1000000 and row['OUTSTANDING_BAL_ALL_CURRENT'] > 1000000:
            row['OUTSTANDING_BAL_CC_CURRENT'] = row['OUTSTANDING_BAL_CC_CURRENT'] if pd.notna(row['OUTSTANDING_BAL_CC_CURRENT']) else (row['OUTSTANDING_BAL_ALL_CURRENT'] - (row['OUTSTANDING_BAL_LOAN_CURRENT'] - 1000000))

    return row

credit_data = credit_data.apply(fill_outstanding_bal, axis=1)



NOTE: ĐỐI VỚI CÁC CỘT INCREASING_BAL_3M_ALL, INCREASING_BAL_3M_LOAN, INCREASING_BAL_3M_CC, INCREASING_BAL_6M_ALL, INCREASING_BAL_6M_LOAN, INCREASING_BAL_6M_CC,
                        ENQUIRIES_3M_6M, ENQUIRIES_FROM_BANK_3M_6M, ENQUIRIES_FROM_NON_BANK_3M_6M,
                        ENQUIRIES_6M_9M, ENQUIRIES_FROM_BANK_6M_9M, ENQUIRIES_FROM_NON_BANK_6M_9M,
                        ENQUIRIES_9M_12M, ENQUIRIES_FROM_BANK_9M_12M, ENQUIRIES_FROM_NON_BANK_9M_12M,
                        ENQUIRIES_6M_12M, ENQUIRIES_FROM_BANK_6M_12M, ENQUIRIES_FROM_NON_BANK_6M_12M,
                        ENQUIRIES_3M_12M, ENQUIRIES_FROM_BANK_3M_12M, ENQUIRIES_FROM_NON_BANK_3M_12M,
                        CREDIT_CARD_MONTH_SINCE_10DPD	CREDIT_CARD_MONTH_SINCE_30DPD	CREDIT_CARD_MONTH_SINCE_60DPD	CREDIT_CARD_MONTH_SINCE_90DPD
    
THÌ NÊN DÙNG most_frequent
                        

CREDIT_CARD_NUMBER_OF_LATE_PAYMENT CHƯA XÁC ĐỊNH ĐƯỢC

So sánh các phương pháp điền giá trị vào các phần có giá trị null để xét xem phương pháp nào đem lại hiệu quả cao nhất

In [12]:
X = credit_data.drop(columns=["customer_id", "label"])
y = credit_data["label"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
imputers = {
    'Mean Imputation': SimpleImputer(strategy='mean'),
    'Median Imputation': SimpleImputer(strategy='median'),
    'Most Frequent Imputation': SimpleImputer(strategy='most_frequent'),
    'KNN Imputation': KNNImputer(n_neighbors=5),
    'Iterative Imputation': IterativeImputer(max_iter=10, random_state=42)
}
results = {}
for name, imputer in imputers.items():
    X_train_imputed = imputer.fit_transform(X_train)
    X_test_imputed = imputer.transform(X_test)
    model = RandomForestClassifier(random_state=42)
    score = cross_val_score(model, X_train_imputed, y_train, cv=5) 
    results[name] = score.mean()
for name, score in results.items():
    print(f"{name}: {score:.4f}")

Mean Imputation: 0.8924
Median Imputation: 0.8922
Most Frequent Imputation: 0.8921
KNN Imputation: 0.8916
Iterative Imputation: 0.8917


In [16]:
max_imputer = max(results, key=results.get)
max_score = results[max_imputer]
print(f"Phương pháp Impute có số điểm trung bình cao nhất: {max_imputer}: {max_score:.4f}")

Phương pháp Impute có số điểm trung bình cao nhất: Mean Imputation: 0.8924


Có thể thấy phương pháp Mean Imputation sẽ cho ra mô hình có hiệu suất cao nhất. Vậy nên sẽ dùng phương pháp này để điền giá trị vào các giá trị null.

In [21]:
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
imputer.fit(credit_data)
imputed_credit_data = imputer.transform(credit_data)
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
imputer.fit(credit_data)
imputed_credit_data = imputer.transform(credit_data)

,customer_id,label,SHORT_TERM_COUNT,MID_TERM_COUNT,LONG_TERM_COUNT,SHORT_TERM_COUNT_BANK,MID_TERM_COUNT_BANK,LONG_TERM_COUNT_BANK,SHORT_TERM_COUNT_NON_BANK,MID_TERM_COUNT_NON_BANK,LONG_TERM_COUNT_NON_BANK,NUMBER_OF_LOANS,NUMBER_OF_LOANS_BANK,NUMBER_OF_LOANS_NON_BANK,NUMBER_OF_CREDIT_CARDS,NUMBER_OF_CREDIT_CARDS_BANK,NUMBER_OF_CREDIT_CARDS_NON_BANK,NUMBER_OF_RELATIONSHIP,NUMBER_OF_RELATIONSHIP_BANK,NUMBER_OF_RELATIONSHIP_NON_BANK,NUM_NEW_LOAN_TAKEN_3M,NUM_NEW_LOAN_TAKEN_6M,NUM_NEW_LOAN_TAKEN_9M,NUM_NEW_LOAN_TAKEN_12M,NUM_NEW_LOAN_TAKEN_BANK_3M,NUM_NEW_LOAN_TAKEN_BANK_6M,NUM_NEW_LOAN_TAKEN_BANK_9M,NUM_NEW_LOAN_TAKEN_BANK_12M,NUM_NEW_LOAN_TAKEN_NON_BANK_3M,NUM_NEW_LOAN_TAKEN_NON_BANK_6M,NUM_NEW_LOAN_TAKEN_NON_BANK_9M,NUM_NEW_LOAN_TAKEN_NON_BANK_12M,OUTSTANDING_BAL_LOAN_CURRENT,OUTSTANDING_BAL_LOAN_3M,OUTSTANDING_BAL_LOAN_6M,OUTSTANDING_BAL_LOAN_9M,OUTSTANDING_BAL_LOAN_12M,OUTSTANDING_BAL_CC_3M,OUTSTANDING_BAL_CC_6M,OUTSTANDING_BAL_CC_9M,OUTSTANDING_BAL_CC_12M,OUTSTANDING_BAL_ALL_3M,OUTSTANDING_BAL_ALL_6M,OUTSTANDING_BAL_ALL_9M,OUTSTANDING_BAL_ALL_12M,OUTSTANDING_BAL_LOAN_3M_6M,OUTSTANDING_BAL_LOAN_6M_9M,OUTSTANDING_BAL_LOAN_9M_12M,OUTSTANDING_BAL_LOAN_6M_12M,OUTSTANDING_BAL_LOAN_3M_12M,OUTSTANDING_BAL_CC_3M_6M,OUTSTANDING_BAL_CC_6M_9M,OUTSTANDING_BAL_CC_9M_12M,OUTSTANDING_BAL_CC_6M_12M,OUTSTANDING_BAL_CC_3M_12M,OUTSTANDING_BAL_ALL_3M_6M,OUTSTANDING_BAL_ALL_6M_9M,OUTSTANDING_BAL_ALL_9M_12M,OUTSTANDING_BAL_ALL_6M_12M,OUTSTANDING_BAL_ALL_3M_12M,INCREASING_BAL_3M_LOAN,INCREASING_BAL_6M_LOAN,INCREASING_BAL_3M_CC,INCREASING_BAL_6M_CC,INCREASING_BAL_3M_ALL,INCREASING_BAL_6M_ALL,OUTSTANDING_BAL_CC_CURRENT,CREDIT_CARD_MONTH_SINCE_10DPD,CREDIT_CARD_MONTH_SINCE_30DPD,CREDIT_CARD_MONTH_SINCE_60DPD,CREDIT_CARD_MONTH_SINCE_90DPD,CREDIT_CARD_NUMBER_OF_LATE_PAYMENT,ENQUIRIES_3M,ENQUIRIES_6M,ENQUIRIES_9M,ENQUIRIES_12M,ENQUIRIES_FROM_BANK_3M,ENQUIRIES_FROM_NON_BANK_3M,ENQUIRIES_FOR_LOAN_3M,ENQUIRIES_FOR_CC_3M,ENQUIRIES_FROM_BANK_FOR_LOAN_3M,ENQUIRIES_FROM_NON_BANK_FOR_LOAN_3M,ENQUIRIES_FROM_BANK_FOR_CC_3M,ENQUIRIES_FROM_NON_BANK_FOR_CC_3M,ENQUIRIES_FROM_BANK_6M,ENQUIRIES_FROM_NON_BANK_6M,ENQUIRIES_FOR_LOAN_6M,ENQUIRIES_FOR_CC_6M,ENQUIRIES_FROM_BANK_FOR_LOAN_6M,ENQUIRIES_FROM_NON_BANK_FOR_LOAN_6M,ENQUIRIES_FROM_BANK_FOR_CC_6M,ENQUIRIES_FROM_NON_BANK_FOR_CC_6M,ENQUIRIES_FROM_BANK_9M,ENQUIRIES_FROM_NON_BANK_9M,ENQUIRIES_FOR_LOAN_9M,ENQUIRIES_FOR_CC_9M,ENQUIRIES_FROM_BANK_FOR_LOAN_9M,ENQUIRIES_FROM_NON_BANK_FOR_LOAN_9M,ENQUIRIES_FROM_BANK_FOR_CC_9M,ENQUIRIES_FROM_NON_BANK_FOR_CC_9M,ENQUIRIES_FROM_BANK_12M,ENQUIRIES_FROM_NON_BANK_12M,ENQUIRIES_FOR_LOAN_12M,ENQUIRIES_FOR_CC_12M,ENQUIRIES_FROM_BANK_FOR_LOAN_12M,ENQUIRIES_FROM_NON_BANK_FOR_LOAN_12M,ENQUIRIES_FROM_BANK_FOR_CC_12M,ENQUIRIES_FROM_NON_BANK_FOR_CC_12M,ENQUIRIES_3M_6M,ENQUIRIES_6M_9M,ENQUIRIES_9M_12M,ENQUIRIES_6M_12M,ENQUIRIES_3M_12M,ENQUIRIES_FROM_BANK_3M_6M,ENQUIRIES_FROM_BANK_6M_9M,ENQUIRIES_FROM_BANK_9M_12M,ENQUIRIES_FROM_BANK_6M_12M,ENQUIRIES_FROM_BANK_3M_12M,ENQUIRIES_FROM_NON_BANK_3M_6M,ENQUIRIES_FROM_NON_BANK_6M_9M,ENQUIRIES_FROM_NON_BANK_9M_12M,ENQUIRIES_FROM_NON_BANK_6M_12M,ENQUIRIES_FROM_NON_BANK_3M_12M,OUTSTANDING_BAL_ALL_CURRENT
0,"1,639.00",1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,6.65,1.00,1.00,1.00,1.00,1.00,1.00,1.00,6.66,1.00,"1,000,000.00","1,000,000.00","1,000,000.00","1,000,000.00","1,000,205.00","1,000,000.00","1,000,000.00","1,000,000.00","1,000,163.84","1,000,000.00","1,000,000.00","1,000,000.00","1,000,205.00","1,000,000.00","1,000,000.00","1,000,000.00","1,000,000.00","1,000,000.00","1,000,000.00","1,000,000.00","1,000,000.00","1,000,000.00","1,000,000.00","1,000,000.00","1,000,000.00","1,000,000.00","1,000,000.00","1,000,000.00","1,000,000.00","1,000,000.00","1,000,000.00","1,000,000.00","1,000,000.00","1,000,000.33","1,000,000.00",431.00,431.00,431.00,431.00,1.00,7.00,7.00,17.00,17.00,2.00,7.00,7.00,2.00,2.00,7.00,2.00,2.00,2.00,7.00,7.00,2.00,2.00,7.00,2.00,2.56,2.00,17.00,12.00,7.00,2.00,12.00,2.00,7.0

In [24]:
imputed_credit_data.isnull().any()

customer_id                       False
label                             False
SHORT_TERM_COUNT                  False
MID_TERM_COUNT                    False
LONG_TERM_COUNT                   False
                                  ...  
ENQUIRIES_FROM_NON_BANK_6M_9M     False
ENQUIRIES_FROM_NON_BANK_9M_12M    False
ENQUIRIES_FROM_NON_BANK_6M_12M    False
ENQUIRIES_FROM_NON_BANK_3M_12M    False
OUTSTANDING_BAL_ALL_CURRENT       False
Length: 124, dtype: bool

Như vậy dataset đã không còn giá trị null, những giá trị null trước đó đã được thay thế bằng giá trị trung bình của cột

In [31]:
credit_data = imputed_credit_data

Xác định các giá trị ngoại lai ở mỗi cột

In [38]:
outlier_values = {}

for c in remaining_columns:
    Q1 = credit_data[c].quantile(0.25)
    Q3 = credit_data[c].quantile(0.75)
    IQR = Q3 - Q1
    # Ngưỡng ngoại lai
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    #Lưu giá trị ngoại lai
    outlier_values[c] = credit_data[(credit_data[c] < lower_bound) | (credit_data[c] > upper_bound)][c]

for col, outlier in outlier_values.items():
    print(f"Giá trị ngoại lai cho cột {col}:")
    print(outlier.unique())
    print("\n")


Giá trị ngoại lai cho cột SHORT_TERM_COUNT:
[19. 13. 16. 22. 25.]


Giá trị ngoại lai cho cột MID_TERM_COUNT:
[10. 13. 16. 22. 19. 25.]


Giá trị ngoại lai cho cột LONG_TERM_COUNT:
[1.06333333 4.         7.        ]


Giá trị ngoại lai cho cột SHORT_TERM_COUNT_BANK:
[10. 13.]


Giá trị ngoại lai cho cột MID_TERM_COUNT_BANK:
[ 1.6105  4.      7.     10.     13.     16.     22.    ]


Giá trị ngoại lai cho cột LONG_TERM_COUNT_BANK:
[1.06133333 4.         7.        ]


Giá trị ngoại lai cho cột SHORT_TERM_COUNT_NON_BANK:
[13. 10. 16.]


Giá trị ngoại lai cho cột MID_TERM_COUNT_NON_BANK:
[10. 16. 13. 19.]


Giá trị ngoại lai cho cột LONG_TERM_COUNT_NON_BANK:
[1.00233333 4.         7.        ]


Giá trị ngoại lai cho cột NUMBER_OF_LOANS:
[22. 25. 28. 31. 34.]


Giá trị ngoại lai cho cột NUMBER_OF_LOANS_BANK:
[10. 16. 13. 19. 25. 22.]


Giá trị ngoại lai cho cột NUMBER_OF_LOANS_NON_BANK:
[13. 16. 19. 22.]


Giá trị ngoại lai cho cột NUMBER_OF_CREDIT_CARDS:
[19. 55. 25. 31. 22. 49. 34. 28. 40

In [44]:
outlier_records = {}

# Xác định và lưu các bản ghi ngoại lai cho mỗi cột
for c in remaining_columns:
    Q1 = credit_data[c].quantile(0.25)
    Q3 = credit_data[c].quantile(0.75)
    IQR = Q3 - Q1
    
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    outlier_records = credit_data[(credit_data[c] < lower_bound) | (credit_data[c] > upper_bound)]

outlier_records.head(10)


,customer_id,label,SHORT_TERM_COUNT,MID_TERM_COUNT,LONG_TERM_COUNT,SHORT_TERM_COUNT_BANK,MID_TERM_COUNT_BANK,LONG_TERM_COUNT_BANK,SHORT_TERM_COUNT_NON_BANK,MID_TERM_COUNT_NON_BANK,LONG_TERM_COUNT_NON_BANK,NUMBER_OF_LOANS,NUMBER_OF_LOANS_BANK,NUMBER_OF_LOANS_NON_BANK,NUMBER_OF_CREDIT_CARDS,NUMBER_OF_CREDIT_CARDS_BANK,NUMBER_OF_CREDIT_CARDS_NON_BANK,NUMBER_OF_RELATIONSHIP,NUMBER_OF_RELATIONSHIP_BANK,NUMBER_OF_RELATIONSHIP_NON_BANK,NUM_NEW_LOAN_TAKEN_3M,NUM_NEW_LOAN_TAKEN_6M,NUM_NEW_LOAN_TAKEN_9M,NUM_NEW_LOAN_TAKEN_12M,NUM_NEW_LOAN_TAKEN_BANK_3M,NUM_NEW_LOAN_TAKEN_BANK_6M,NUM_NEW_LOAN_TAKEN_BANK_9M,NUM_NEW_LOAN_TAKEN_BANK_12M,NUM_NEW_LOAN_TAKEN_NON_BANK_3M,NUM_NEW_LOAN_TAKEN_NON_BANK_6M,NUM_NEW_LOAN_TAKEN_NON_BANK_9M,NUM_NEW_LOAN_TAKEN_NON_BANK_12M,OUTSTANDING_BAL_LOAN_CURRENT,OUTSTANDING_BAL_LOAN_3M,OUTSTANDING_BAL_LOAN_6M,OUTSTANDING_BAL_LOAN_9M,OUTSTANDING_BAL_LOAN_12M,OUTSTANDING_BAL_CC_3M,OUTSTANDING_BAL_CC_6M,OUTSTANDING_BAL_CC_9M,OUTSTANDING_BAL_CC_12M,OUTSTANDING_BAL_ALL_3M,OUTSTANDING_BAL_ALL_6M,OUTSTANDING_BAL_ALL_9M,OUTSTANDING_BAL_ALL_12M,OUTSTANDING_BAL_LOAN_3M_6M,OUTSTANDING_BAL_LOAN_6M_9M,OUTSTANDING_BAL_LOAN_9M_12M,OUTSTANDING_BAL_LOAN_6M_12M,OUTSTANDING_BAL_LOAN_3M_12M,OUTSTANDING_BAL_CC_3M_6M,OUTSTANDING_BAL_CC_6M_9M,OUTSTANDING_BAL_CC_9M_12M,OUTSTANDING_BAL_CC_6M_12M,OUTSTANDING_BAL_CC_3M_12M,OUTSTANDING_BAL_ALL_3M_6M,OUTSTANDING_BAL_ALL_6M_9M,OUTSTANDING_BAL_ALL_9M_12M,OUTSTANDING_BAL_ALL_6M_12M,OUTSTANDING_BAL_ALL_3M_12M,INCREASING_BAL_3M_LOAN,INCREASING_BAL_6M_LOAN,INCREASING_BAL_3M_CC,INCREASING_BAL_6M_CC,INCREASING_BAL_3M_ALL,INCREASING_BAL_6M_ALL,OUTSTANDING_BAL_CC_CURRENT,CREDIT_CARD_MONTH_SINCE_10DPD,CREDIT_CARD_MONTH_SINCE_30DPD,CREDIT_CARD_MONTH_SINCE_60DPD,CREDIT_CARD_MONTH_SINCE_90DPD,CREDIT_CARD_NUMBER_OF_LATE_PAYMENT,ENQUIRIES_3M,ENQUIRIES_6M,ENQUIRIES_9M,ENQUIRIES_12M,ENQUIRIES_FROM_BANK_3M,ENQUIRIES_FROM_NON_BANK_3M,ENQUIRIES_FOR_LOAN_3M,ENQUIRIES_FOR_CC_3M,ENQUIRIES_FROM_BANK_FOR_LOAN_3M,ENQUIRIES_FROM_NON_BANK_FOR_LOAN_3M,ENQUIRIES_FROM_BANK_FOR_CC_3M,ENQUIRIES_FROM_NON_BANK_FOR_CC_3M,ENQUIRIES_FROM_BANK_6M,ENQUIRIES_FROM_NON_BANK_6M,ENQUIRIES_FOR_LOAN_6M,ENQUIRIES_FOR_CC_6M,ENQUIRIES_FROM_BANK_FOR_LOAN_6M,ENQUIRIES_FROM_NON_BANK_FOR_LOAN_6M,ENQUIRIES_FROM_BANK_FOR_CC_6M,ENQUIRIES_FROM_NON_BANK_FOR_CC_6M,ENQUIRIES_FROM_BANK_9M,ENQUIRIES_FROM_NON_BANK_9M,ENQUIRIES_FOR_LOAN_9M,ENQUIRIES_FOR_CC_9M,ENQUIRIES_FROM_BANK_FOR_LOAN_9M,ENQUIRIES_FROM_NON_BANK_FOR_LOAN_9M,ENQUIRIES_FROM_BANK_FOR_CC_9M,ENQUIRIES_FROM_NON_BANK_FOR_CC_9M,ENQUIRIES_FROM_BANK_12M,ENQUIRIES_FROM_NON_BANK_12M,ENQUIRIES_FOR_LOAN_12M,ENQUIRIES_FOR_CC_12M,ENQUIRIES_FROM_BANK_FOR_LOAN_12M,ENQUIRIES_FROM_NON_BANK_FOR_LOAN_12M,ENQUIRIES_FROM_BANK_FOR_CC_12M,ENQUIRIES_FROM_NON_BANK_FOR_CC_12M,ENQUIRIES_3M_6M,ENQUIRIES_6M_9M,ENQUIRIES_9M_12M,ENQUIRIES_6M_12M,ENQUIRIES_3M_12M,ENQUIRIES_FROM_BANK_3M_6M,ENQUIRIES_FROM_BANK_6M_9M,ENQUIRIES_FROM_BANK_9M_12M,ENQUIRIES_FROM_BANK_6M_12M,ENQUIRIES_FROM_BANK_3M_12M,ENQUIRIES_FROM_NON_BANK_3M_6M,ENQUIRIES_FROM_NON_BANK_6M_9M,ENQUIRIES_FROM_NON_BANK_9M_12M,ENQUIRIES_FROM_NON_BANK_6M_12M,ENQUIRIES_FROM_NON_BANK_3M_12M,OUTSTANDING_BAL_ALL_CURRENT
6,"20,339.00",0.00,4.00,10.00,1.06,1.00,4.00,1.00,4.00,7.00,1.00,13.00,4.00,10.00,5.29,13.00,4.00,28.00,16.00,13.00,4.00,5.50,6.65,7.00,4.00,4.00,7.00,7.00,4.00,4.00,7.00,7.00,"1,001,670.00","1,001,053.33","1,001,286.67","1,001,291.11","1,001,293.33","1,000,380.00","1,000,565.00","1,000,174.69","1,000,568.33","1,000,791.74","1,001,851.67","1,001,862.22","1,001,861.67","1,000,008.19","1,000,009.97","1,000,009.98","1,000,009.95","1,000,008.14","1,000,004.25","1,000,009.89","1,000,010.05","1,000,009.94","1,000,006.69","1,000,007.74","1,000,009.94","1,000,010.00","1,000,012.84","1,000,015.76","1,000,000.00","1,000,000.00","1,000,000.00","999,998.21","1,000,001.42","1,000,000.00","1,001,070.00",431.00,431.00,431.00,431.00,1.00,17.00,67.00,97.00,97.00,12.00,9.97,12.00,2.00,7.00,7.00,2.00,2.00,47.00,22.00,37.00,27.00,22.00,10.94,22.00,7.00,77.00,22.00,52.00,42.00,

In [46]:
credit_data=credit_data.drop(columns=["customer_id"])
credit_data

,label,SHORT_TERM_COUNT,MID_TERM_COUNT,LONG_TERM_COUNT,SHORT_TERM_COUNT_BANK,MID_TERM_COUNT_BANK,LONG_TERM_COUNT_BANK,SHORT_TERM_COUNT_NON_BANK,MID_TERM_COUNT_NON_BANK,LONG_TERM_COUNT_NON_BANK,NUMBER_OF_LOANS,NUMBER_OF_LOANS_BANK,NUMBER_OF_LOANS_NON_BANK,NUMBER_OF_CREDIT_CARDS,NUMBER_OF_CREDIT_CARDS_BANK,NUMBER_OF_CREDIT_CARDS_NON_BANK,NUMBER_OF_RELATIONSHIP,NUMBER_OF_RELATIONSHIP_BANK,NUMBER_OF_RELATIONSHIP_NON_BANK,NUM_NEW_LOAN_TAKEN_3M,NUM_NEW_LOAN_TAKEN_6M,NUM_NEW_LOAN_TAKEN_9M,NUM_NEW_LOAN_TAKEN_12M,NUM_NEW_LOAN_TAKEN_BANK_3M,NUM_NEW_LOAN_TAKEN_BANK_6M,NUM_NEW_LOAN_TAKEN_BANK_9M,NUM_NEW_LOAN_TAKEN_BANK_12M,NUM_NEW_LOAN_TAKEN_NON_BANK_3M,NUM_NEW_LOAN_TAKEN_NON_BANK_6M,NUM_NEW_LOAN_TAKEN_NON_BANK_9M,NUM_NEW_LOAN_TAKEN_NON_BANK_12M,OUTSTANDING_BAL_LOAN_CURRENT,OUTSTANDING_BAL_LOAN_3M,OUTSTANDING_BAL_LOAN_6M,OUTSTANDING_BAL_LOAN_9M,OUTSTANDING_BAL_LOAN_12M,OUTSTANDING_BAL_CC_3M,OUTSTANDING_BAL_CC_6M,OUTSTANDING_BAL_CC_9M,OUTSTANDING_BAL_CC_12M,OUTSTANDING_BAL_ALL_3M,OUTSTANDING_BAL_ALL_6M,OUTSTANDING_BAL_ALL_9M,OUTSTANDING_BAL_ALL_12M,OUTSTANDING_BAL_LOAN_3M_6M,OUTSTANDING_BAL_LOAN_6M_9M,OUTSTANDING_BAL_LOAN_9M_12M,OUTSTANDING_BAL_LOAN_6M_12M,OUTSTANDING_BAL_LOAN_3M_12M,OUTSTANDING_BAL_CC_3M_6M,OUTSTANDING_BAL_CC_6M_9M,OUTSTANDING_BAL_CC_9M_12M,OUTSTANDING_BAL_CC_6M_12M,OUTSTANDING_BAL_CC_3M_12M,OUTSTANDING_BAL_ALL_3M_6M,OUTSTANDING_BAL_ALL_6M_9M,OUTSTANDING_BAL_ALL_9M_12M,OUTSTANDING_BAL_ALL_6M_12M,OUTSTANDING_BAL_ALL_3M_12M,INCREASING_BAL_3M_LOAN,INCREASING_BAL_6M_LOAN,INCREASING_BAL_3M_CC,INCREASING_BAL_6M_CC,INCREASING_BAL_3M_ALL,INCREASING_BAL_6M_ALL,OUTSTANDING_BAL_CC_CURRENT,CREDIT_CARD_MONTH_SINCE_10DPD,CREDIT_CARD_MONTH_SINCE_30DPD,CREDIT_CARD_MONTH_SINCE_60DPD,CREDIT_CARD_MONTH_SINCE_90DPD,CREDIT_CARD_NUMBER_OF_LATE_PAYMENT,ENQUIRIES_3M,ENQUIRIES_6M,ENQUIRIES_9M,ENQUIRIES_12M,ENQUIRIES_FROM_BANK_3M,ENQUIRIES_FROM_NON_BANK_3M,ENQUIRIES_FOR_LOAN_3M,ENQUIRIES_FOR_CC_3M,ENQUIRIES_FROM_BANK_FOR_LOAN_3M,ENQUIRIES_FROM_NON_BANK_FOR_LOAN_3M,ENQUIRIES_FROM_BANK_FOR_CC_3M,ENQUIRIES_FROM_NON_BANK_FOR_CC_3M,ENQUIRIES_FROM_BANK_6M,ENQUIRIES_FROM_NON_BANK_6M,ENQUIRIES_FOR_LOAN_6M,ENQUIRIES_FOR_CC_6M,ENQUIRIES_FROM_BANK_FOR_LOAN_6M,ENQUIRIES_FROM_NON_BANK_FOR_LOAN_6M,ENQUIRIES_FROM_BANK_FOR_CC_6M,ENQUIRIES_FROM_NON_BANK_FOR_CC_6M,ENQUIRIES_FROM_BANK_9M,ENQUIRIES_FROM_NON_BANK_9M,ENQUIRIES_FOR_LOAN_9M,ENQUIRIES_FOR_CC_9M,ENQUIRIES_FROM_BANK_FOR_LOAN_9M,ENQUIRIES_FROM_NON_BANK_FOR_LOAN_9M,ENQUIRIES_FROM_BANK_FOR_CC_9M,ENQUIRIES_FROM_NON_BANK_FOR_CC_9M,ENQUIRIES_FROM_BANK_12M,ENQUIRIES_FROM_NON_BANK_12M,ENQUIRIES_FOR_LOAN_12M,ENQUIRIES_FOR_CC_12M,ENQUIRIES_FROM_BANK_FOR_LOAN_12M,ENQUIRIES_FROM_NON_BANK_FOR_LOAN_12M,ENQUIRIES_FROM_BANK_FOR_CC_12M,ENQUIRIES_FROM_NON_BANK_FOR_CC_12M,ENQUIRIES_3M_6M,ENQUIRIES_6M_9M,ENQUIRIES_9M_12M,ENQUIRIES_6M_12M,ENQUIRIES_3M_12M,ENQUIRIES_FROM_BANK_3M_6M,ENQUIRIES_FROM_BANK_6M_9M,ENQUIRIES_FROM_BANK_9M_12M,ENQUIRIES_FROM_BANK_6M_12M,ENQUIRIES_FROM_BANK_3M_12M,ENQUIRIES_FROM_NON_BANK_3M_6M,ENQUIRIES_FROM_NON_BANK_6M_9M,ENQUIRIES_FROM_NON_BANK_9M_12M,ENQUIRIES_FROM_NON_BANK_6M_12M,ENQUIRIES_FROM_NON_BANK_3M_12M,OUTSTANDING_BAL_ALL_CURRENT
0,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,6.65,1.00,1.00,1.00,1.00,1.00,1.00,1.00,6.66,1.00,"1,000,000.00","1,000,000.00","1,000,000.00","1,000,000.00","1,000,205.00","1,000,000.00","1,000,000.00","1,000,000.00","1,000,163.84","1,000,000.00","1,000,000.00","1,000,000.00","1,000,205.00","1,000,000.00","1,000,000.00","1,000,000.00","1,000,000.00","1,000,000.00","1,000,000.00","1,000,000.00","1,000,000.00","1,000,000.00","1,000,000.00","1,000,000.00","1,000,000.00","1,000,000.00","1,000,000.00","1,000,000.00","1,000,000.00","1,000,000.00","1,000,000.00","1,000,000.00","1,000,000.00","1,000,000.33","1,000,000.00",431.00,431.00,431.00,431.00,1.00,7.00,7.00,17.00,17.00,2.00,7.00,7.00,2.00,2.00,7.00,2.00,2.00,2.00,7.00,7.00,2.00,2.00,7.00,2.00,2.56,2.00,17.00,12.00,7.00,2.00,12.00,2.00,7.00,2.00,17.00,12.00,7.00

Loại bỏ các bản ghi chứa ngoại lai

In [47]:
for c in remaining_columns:
    Q1 = credit_data[c].quantile(0.25)
    Q3 = credit_data[c].quantile(0.75)
    IQR = Q3 - Q1
    
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    mask = (credit_data[c] >= lower_bound) & (credit_data[c] <= upper_bound)

# Lọc DataFrame để chỉ giữ lại các bản ghi không có ngoại lai
cleaned_credit_data = credit_data[mask]

print(f"Số lượng bản ghi ban đầu: {len(credit_data)}")
print(f"Số lượng bản ghi sau khi loại bỏ ngoại lai: {len(cleaned_credit_data)}")


Số lượng bản ghi ban đầu: 20000
Số lượng bản ghi sau khi loại bỏ ngoại lai: 18772


In [50]:
cleaned_credit_data.head(10)

,label,SHORT_TERM_COUNT,MID_TERM_COUNT,LONG_TERM_COUNT,SHORT_TERM_COUNT_BANK,MID_TERM_COUNT_BANK,LONG_TERM_COUNT_BANK,SHORT_TERM_COUNT_NON_BANK,MID_TERM_COUNT_NON_BANK,LONG_TERM_COUNT_NON_BANK,NUMBER_OF_LOANS,NUMBER_OF_LOANS_BANK,NUMBER_OF_LOANS_NON_BANK,NUMBER_OF_CREDIT_CARDS,NUMBER_OF_CREDIT_CARDS_BANK,NUMBER_OF_CREDIT_CARDS_NON_BANK,NUMBER_OF_RELATIONSHIP,NUMBER_OF_RELATIONSHIP_BANK,NUMBER_OF_RELATIONSHIP_NON_BANK,NUM_NEW_LOAN_TAKEN_3M,NUM_NEW_LOAN_TAKEN_6M,NUM_NEW_LOAN_TAKEN_9M,NUM_NEW_LOAN_TAKEN_12M,NUM_NEW_LOAN_TAKEN_BANK_3M,NUM_NEW_LOAN_TAKEN_BANK_6M,NUM_NEW_LOAN_TAKEN_BANK_9M,NUM_NEW_LOAN_TAKEN_BANK_12M,NUM_NEW_LOAN_TAKEN_NON_BANK_3M,NUM_NEW_LOAN_TAKEN_NON_BANK_6M,NUM_NEW_LOAN_TAKEN_NON_BANK_9M,NUM_NEW_LOAN_TAKEN_NON_BANK_12M,OUTSTANDING_BAL_LOAN_CURRENT,OUTSTANDING_BAL_LOAN_3M,OUTSTANDING_BAL_LOAN_6M,OUTSTANDING_BAL_LOAN_9M,OUTSTANDING_BAL_LOAN_12M,OUTSTANDING_BAL_CC_3M,OUTSTANDING_BAL_CC_6M,OUTSTANDING_BAL_CC_9M,OUTSTANDING_BAL_CC_12M,OUTSTANDING_BAL_ALL_3M,OUTSTANDING_BAL_ALL_6M,OUTSTANDING_BAL_ALL_9M,OUTSTANDING_BAL_ALL_12M,OUTSTANDING_BAL_LOAN_3M_6M,OUTSTANDING_BAL_LOAN_6M_9M,OUTSTANDING_BAL_LOAN_9M_12M,OUTSTANDING_BAL_LOAN_6M_12M,OUTSTANDING_BAL_LOAN_3M_12M,OUTSTANDING_BAL_CC_3M_6M,OUTSTANDING_BAL_CC_6M_9M,OUTSTANDING_BAL_CC_9M_12M,OUTSTANDING_BAL_CC_6M_12M,OUTSTANDING_BAL_CC_3M_12M,OUTSTANDING_BAL_ALL_3M_6M,OUTSTANDING_BAL_ALL_6M_9M,OUTSTANDING_BAL_ALL_9M_12M,OUTSTANDING_BAL_ALL_6M_12M,OUTSTANDING_BAL_ALL_3M_12M,INCREASING_BAL_3M_LOAN,INCREASING_BAL_6M_LOAN,INCREASING_BAL_3M_CC,INCREASING_BAL_6M_CC,INCREASING_BAL_3M_ALL,INCREASING_BAL_6M_ALL,OUTSTANDING_BAL_CC_CURRENT,CREDIT_CARD_MONTH_SINCE_10DPD,CREDIT_CARD_MONTH_SINCE_30DPD,CREDIT_CARD_MONTH_SINCE_60DPD,CREDIT_CARD_MONTH_SINCE_90DPD,CREDIT_CARD_NUMBER_OF_LATE_PAYMENT,ENQUIRIES_3M,ENQUIRIES_6M,ENQUIRIES_9M,ENQUIRIES_12M,ENQUIRIES_FROM_BANK_3M,ENQUIRIES_FROM_NON_BANK_3M,ENQUIRIES_FOR_LOAN_3M,ENQUIRIES_FOR_CC_3M,ENQUIRIES_FROM_BANK_FOR_LOAN_3M,ENQUIRIES_FROM_NON_BANK_FOR_LOAN_3M,ENQUIRIES_FROM_BANK_FOR_CC_3M,ENQUIRIES_FROM_NON_BANK_FOR_CC_3M,ENQUIRIES_FROM_BANK_6M,ENQUIRIES_FROM_NON_BANK_6M,ENQUIRIES_FOR_LOAN_6M,ENQUIRIES_FOR_CC_6M,ENQUIRIES_FROM_BANK_FOR_LOAN_6M,ENQUIRIES_FROM_NON_BANK_FOR_LOAN_6M,ENQUIRIES_FROM_BANK_FOR_CC_6M,ENQUIRIES_FROM_NON_BANK_FOR_CC_6M,ENQUIRIES_FROM_BANK_9M,ENQUIRIES_FROM_NON_BANK_9M,ENQUIRIES_FOR_LOAN_9M,ENQUIRIES_FOR_CC_9M,ENQUIRIES_FROM_BANK_FOR_LOAN_9M,ENQUIRIES_FROM_NON_BANK_FOR_LOAN_9M,ENQUIRIES_FROM_BANK_FOR_CC_9M,ENQUIRIES_FROM_NON_BANK_FOR_CC_9M,ENQUIRIES_FROM_BANK_12M,ENQUIRIES_FROM_NON_BANK_12M,ENQUIRIES_FOR_LOAN_12M,ENQUIRIES_FOR_CC_12M,ENQUIRIES_FROM_BANK_FOR_LOAN_12M,ENQUIRIES_FROM_NON_BANK_FOR_LOAN_12M,ENQUIRIES_FROM_BANK_FOR_CC_12M,ENQUIRIES_FROM_NON_BANK_FOR_CC_12M,ENQUIRIES_3M_6M,ENQUIRIES_6M_9M,ENQUIRIES_9M_12M,ENQUIRIES_6M_12M,ENQUIRIES_3M_12M,ENQUIRIES_FROM_BANK_3M_6M,ENQUIRIES_FROM_BANK_6M_9M,ENQUIRIES_FROM_BANK_9M_12M,ENQUIRIES_FROM_BANK_6M_12M,ENQUIRIES_FROM_BANK_3M_12M,ENQUIRIES_FROM_NON_BANK_3M_6M,ENQUIRIES_FROM_NON_BANK_6M_9M,ENQUIRIES_FROM_NON_BANK_9M_12M,ENQUIRIES_FROM_NON_BANK_6M_12M,ENQUIRIES_FROM_NON_BANK_3M_12M,OUTSTANDING_BAL_ALL_CURRENT
0,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,6.65,1.00,1.00,1.00,1.00,1.00,1.00,1.00,6.66,1.00,"1,000,000.00","1,000,000.00","1,000,000.00","1,000,000.00","1,000,205.00","1,000,000.00","1,000,000.00","1,000,000.00","1,000,163.84","1,000,000.00","1,000,000.00","1,000,000.00","1,000,205.00","1,000,000.00","1,000,000.00","1,000,000.00","1,000,000.00","1,000,000.00","1,000,000.00","1,000,000.00","1,000,000.00","1,000,000.00","1,000,000.00","1,000,000.00","1,000,000.00","1,000,000.00","1,000,000.00","1,000,000.00","1,000,000.00","1,000,000.00","1,000,000.00","1,000,000.00","1,000,000.00","1,000,000.33","1,000,000.00",431.00,431.00,431.00,431.00,1.00,7.00,7.00,17.00,17.00,2.00,7.00,7.00,2.00,2.00,7.00,2.00,2.00,2.00,7.00,7.00,2.00,2.00,7.00,2.00,2.56,2.00,17.00,12.00,7.00,2.00,12.00,2.00,7.00,2.00,17.00,12.00,7.00

In [51]:
cleaned_credit_data.describe()

,label,SHORT_TERM_COUNT,MID_TERM_COUNT,LONG_TERM_COUNT,SHORT_TERM_COUNT_BANK,MID_TERM_COUNT_BANK,LONG_TERM_COUNT_BANK,SHORT_TERM_COUNT_NON_BANK,MID_TERM_COUNT_NON_BANK,LONG_TERM_COUNT_NON_BANK,NUMBER_OF_LOANS,NUMBER_OF_LOANS_BANK,NUMBER_OF_LOANS_NON_BANK,NUMBER_OF_CREDIT_CARDS,NUMBER_OF_CREDIT_CARDS_BANK,NUMBER_OF_CREDIT_CARDS_NON_BANK,NUMBER_OF_RELATIONSHIP,NUMBER_OF_RELATIONSHIP_BANK,NUMBER_OF_RELATIONSHIP_NON_BANK,NUM_NEW_LOAN_TAKEN_3M,NUM_NEW_LOAN_TAKEN_6M,NUM_NEW_LOAN_TAKEN_9M,NUM_NEW_LOAN_TAKEN_12M,NUM_NEW_LOAN_TAKEN_BANK_3M,NUM_NEW_LOAN_TAKEN_BANK_6M,NUM_NEW_LOAN_TAKEN_BANK_9M,NUM_NEW_LOAN_TAKEN_BANK_12M,NUM_NEW_LOAN_TAKEN_NON_BANK_3M,NUM_NEW_LOAN_TAKEN_NON_BANK_6M,NUM_NEW_LOAN_TAKEN_NON_BANK_9M,NUM_NEW_LOAN_TAKEN_NON_BANK_12M,OUTSTANDING_BAL_LOAN_CURRENT,OUTSTANDING_BAL_LOAN_3M,OUTSTANDING_BAL_LOAN_6M,OUTSTANDING_BAL_LOAN_9M,OUTSTANDING_BAL_LOAN_12M,OUTSTANDING_BAL_CC_3M,OUTSTANDING_BAL_CC_6M,OUTSTANDING_BAL_CC_9M,OUTSTANDING_BAL_CC_12M,OUTSTANDING_BAL_ALL_3M,OUTSTANDING_BAL_ALL_6M,OUTSTANDING_BAL_ALL_9M,OUTSTANDING_BAL_ALL_12M,OUTSTANDING_BAL_LOAN_3M_6M,OUTSTANDING_BAL_LOAN_6M_9M,OUTSTANDING_BAL_LOAN_9M_12M,OUTSTANDING_BAL_LOAN_6M_12M,OUTSTANDING_BAL_LOAN_3M_12M,OUTSTANDING_BAL_CC_3M_6M,OUTSTANDING_BAL_CC_6M_9M,OUTSTANDING_BAL_CC_9M_12M,OUTSTANDING_BAL_CC_6M_12M,OUTSTANDING_BAL_CC_3M_12M,OUTSTANDING_BAL_ALL_3M_6M,OUTSTANDING_BAL_ALL_6M_9M,OUTSTANDING_BAL_ALL_9M_12M,OUTSTANDING_BAL_ALL_6M_12M,OUTSTANDING_BAL_ALL_3M_12M,INCREASING_BAL_3M_LOAN,INCREASING_BAL_6M_LOAN,INCREASING_BAL_3M_CC,INCREASING_BAL_6M_CC,INCREASING_BAL_3M_ALL,INCREASING_BAL_6M_ALL,OUTSTANDING_BAL_CC_CURRENT,CREDIT_CARD_MONTH_SINCE_10DPD,CREDIT_CARD_MONTH_SINCE_30DPD,CREDIT_CARD_MONTH_SINCE_60DPD,CREDIT_CARD_MONTH_SINCE_90DPD,CREDIT_CARD_NUMBER_OF_LATE_PAYMENT,ENQUIRIES_3M,ENQUIRIES_6M,ENQUIRIES_9M,ENQUIRIES_12M,ENQUIRIES_FROM_BANK_3M,ENQUIRIES_FROM_NON_BANK_3M,ENQUIRIES_FOR_LOAN_3M,ENQUIRIES_FOR_CC_3M,ENQUIRIES_FROM_BANK_FOR_LOAN_3M,ENQUIRIES_FROM_NON_BANK_FOR_LOAN_3M,ENQUIRIES_FROM_BANK_FOR_CC_3M,ENQUIRIES_FROM_NON_BANK_FOR_CC_3M,ENQUIRIES_FROM_BANK_6M,ENQUIRIES_FROM_NON_BANK_6M,ENQUIRIES_FOR_LOAN_6M,ENQUIRIES_FOR_CC_6M,ENQUIRIES_FROM_BANK_FOR_LOAN_6M,ENQUIRIES_FROM_NON_BANK_FOR_LOAN_6M,ENQUIRIES_FROM_BANK_FOR_CC_6M,ENQUIRIES_FROM_NON_BANK_FOR_CC_6M,ENQUIRIES_FROM_BANK_9M,ENQUIRIES_FROM_NON_BANK_9M,ENQUIRIES_FOR_LOAN_9M,ENQUIRIES_FOR_CC_9M,ENQUIRIES_FROM_BANK_FOR_LOAN_9M,ENQUIRIES_FROM_NON_BANK_FOR_LOAN_9M,ENQUIRIES_FROM_BANK_FOR_CC_9M,ENQUIRIES_FROM_NON_BANK_FOR_CC_9M,ENQUIRIES_FROM_BANK_12M,ENQUIRIES_FROM_NON_BANK_12M,ENQUIRIES_FOR_LOAN_12M,ENQUIRIES_FOR_CC_12M,ENQUIRIES_FROM_BANK_FOR_LOAN_12M,ENQUIRIES_FROM_NON_BANK_FOR_LOAN_12M,ENQUIRIES_FROM_BANK_FOR_CC_12M,ENQUIRIES_FROM_NON_BANK_FOR_CC_12M,ENQUIRIES_3M_6M,ENQUIRIES_6M_9M,ENQUIRIES_9M_12M,ENQUIRIES_6M_12M,ENQUIRIES_3M_12M,ENQUIRIES_FROM_BANK_3M_6M,ENQUIRIES_FROM_BANK_6M_9M,ENQUIRIES_FROM_BANK_9M_12M,ENQUIRIES_FROM_BANK_6M_12M,ENQUIRIES_FROM_BANK_3M_12M,ENQUIRIES_FROM_NON_BANK_3M_6M,ENQUIRIES_FROM_NON_BANK_6M_9M,ENQUIRIES_FROM_NON_BANK_9M_12M,ENQUIRIES_FROM_NON_BANK_6M_12M,ENQUIRIES_FROM_NON_BANK_3M_12M,OUTSTANDING_BAL_ALL_CURRENT
count,"18,772.00","18,772.00","18,772.00","18,772.00","18,772.00","18,772.00","18,772.00","18,772.00","18,772.00","18,772.00","18,772.00","18,772.00","18,772.00","18,772.00","18,772.00","18,772.00","18,772.00","18,772.00","18,772.00","18,772.00","18,772.00","18,772.00","18,772.00","18,772.00","18,772.00","18,772.00","18,772.00","18,772.00","18,772.00","18,772.00","18,772.00","18,772.00","18,772.00","18,772.00","18,772.00","18,772.00","18,772.00","18,772.00","18,772.00","18,772.00","18,772.00","18,772.00","18,772.00","18,772.00","18,772.00","18,772.00","18,772.00","18,772.00","18,772.00","18,772.00","18,772.00","18,772.00","18,772.00","18,772.00","18,772.00","18,772.00","18,772.00","18,772.00","18,772.00","18,772.00","18,772.00","18,772.00","18,772.00","18,772.00","18,772.00","18,772.00","18,772.00","18,772.00","18,772.00","18,772.00","18,772.00","18,772.00","18,772.00"

In [52]:
cleaned_credit_data.to_csv("clean_dataset.csv",index=False)